<a href="https://colab.research.google.com/github/milad9595naeimi/Revealed_Technological_Advantage/blob/main/Technological_Specialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. from google.colab import drive:

This line imports the drive module from the google.colab package. The google.colab package provides various utilities to interact with Google Colab, and the drive module specifically deals with operations related to Google Drive.

2. drive.mount('/content/drive/'):

This line mounts Google Drive to the specified path in the Colab environment, allowing you to access files stored in your Google Drive as if they were part of the local file system on the Colab instance.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


1. !pip install google-cloud-bigquery:


The actual command pip install google-cloud-bigquery is used to install the Google Cloud BigQuery client library for Python. In a Jupyter notebook or Colab, this line installs the necessary package so you can interact with BigQuery.

2. import os:

This line imports the os module, which provides a way of using operating system-dependent functionality like reading or writing to the file system, environment variables, etc.

3. from google.cloud import bigquery:
This line imports the bigquery module from the google.cloud package, which contains the client library for interacting with BigQuery.

4. import pandas as pd:
This line imports the pandas library, which is a powerful data manipulation and analysis library in Python. It's often used for handling data structures like DataFrames.

In [2]:
#!pip install google-cloud-bigquery
import os
from google.cloud import bigquery
import pandas as pd

1. import os:
This line imports the os module, which provides functions to interact with the operating system, including manipulating environment variables.

2. key_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/patents-426102-ce4b63cd244f.json':
This line defines a variable key_file_path that holds the path to the service account key JSON file. This file is assumed to be uploaded to Google Drive and mounted in Colab.

3. os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_file_path:
This line sets the GOOGLE_APPLICATION_CREDENTIALS environment variable to the path of the service account key file. This environment variable is used by Google Cloud client libraries to authenticate API requests.

In [ ]:
import os

# Assuming the JSON file is uploaded to the root directory in Colab
key_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/patents-426102-ce4b63cd244f.json'

# Set up Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_file_path


1. from google.cloud import bigquery:

This line imports the bigquery module from the google.cloud package. This module provides the functionality needed to interact with Google BigQuery, such as running queries and managing datasets.

2. import pandas as pd:

This line imports the pandas library, which is a powerful tool for data manipulation and analysis in Python. It provides data structures like DataFrames for handling tabular data.

3. import re:

This line imports the re module, which provides support for regular expressions in Python. Regular expressions are used for matching patterns in strings, which can be useful for data cleaning and validation tasks.

4. client = bigquery.Client():

This line initializes a BigQuery client object. The Client class is used to interact with BigQuery, allowing you to run SQL queries, retrieve results, and manage datasets and tables.

In [ ]:
from google.cloud import bigquery
import pandas as pd
import re  # Make sure to import the re module for regular expressions

# Initialize BigQuery client
client = bigquery.Client()


Retrieving data from Google Patents Public Datasets Using BigQuery. Only Change the codes for cpc and it works

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize BigQuery client
client = bigquery.Client()

# Combined SQL query with proper filtering and aggregation
query = """
WITH inventor_data AS (
  SELECT
    publication_number,
    STRING_AGG(inventor_harmonized.country_code, ' | ') AS inventor_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(inventor_harmonized) AS inventor_harmonized
  WHERE
    inventor_harmonized.country_code IN ("AT", "AU", "BE", "CA", "DK", "FI", "DE", "IE", "JP", "NL", "NO", "SE", "CH", "GB", "US")
  GROUP BY
    publication_number
  HAVING
    COUNT(inventor_harmonized.country_code) > 0
),
cpc_data AS (
  SELECT
    publication_number,
    STRING_AGG(DISTINCT cpc.code, ' | ') AS cpc_codes
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(cpc) AS cpc
  WHERE
    cpc.code LIKE "B64B%" OR cpc.code LIKE "B64C%" OR cpc.code LIKE "B64D%" OR cpc.code LIKE "B64F%" OR cpc.code LIKE "B64G%"
  GROUP BY
    publication_number
  HAVING
    COUNT(cpc.code) > 0
)
SELECT
  p.publication_number,
  CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) AS publication_year,
  i.inventor_countries,
  c.cpc_codes
FROM
  `patents-public-data.patents.publications_202310` AS p
JOIN
  inventor_data AS i
ON
  p.publication_number = i.publication_number
JOIN
  cpc_data AS c
ON
  p.publication_number = c.publication_number
WHERE
  LEFT(p.publication_number, 2) = 'US'
  AND CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) BETWEEN 1989 AND 2008
LIMIT 100;
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert to DataFrame
df = results.to_dataframe()

# Display the DataFrame
print(df)


Add "assignee_country" to the previous line of code to have a new column of "assignee_country" in addition to other columns.This code is designed to retrieve patent data from Google BigQuery, specifically focusing on US patents published between 1989 and 2008. It aggregates inventor countries, CPC codes, and assignee countries for each patent, ensuring that the inventor and CPC data meet specified filtering criteria. The final dataset includes publication numbers, publication years, inventor countries, CPC codes, and assignee countries, providing a comprehensive view of the relevant patent information.











1. Detailed Explanation of the Query
Common Table Expressions (CTEs)
inventor_data CTE:

Purpose: To collect and aggregate inventor country codes for each patent.
Details:
""SELECT
  publication_number,
  STRING_AGG(inventor_harmonized.country_code, ' | ') AS inventor_countries
FROM
  `patents-public-data.patents.publications_202310`,
  UNNEST(inventor_harmonized) AS inventor_harmonized
WHERE
  inventor_harmonized.country_code IN ("AT", "AU", "BE", "CA", "DK", "FI", "DE", "IE", "JP", "NL", "NO", "SE", "CH", "GB", "US")
GROUP BY
  publication_number
HAVING
  COUNT(inventor_harmonized.country_code) > 0""

a) UNNEST(inventor_harmonized): Flattens the nested inventor_harmonized array.
b) STRING_AGG(inventor_harmonized.country_code, ' | '): Aggregates inventor country codes separated by |.
c) WHERE clause: Filters inventors by specified country codes.
d) HAVING COUNT(...) > 0: Ensures only patents with at least one valid inventor country code are included.



2. cpc_data CTE:

Purpose: To collect and aggregate CPC codes for each patent.
Details:
""SELECT
  publication_number,
  STRING_AGG(DISTINCT cpc.code, ' | ') AS cpc_codes
FROM
  `patents-public-data.patents.publications_202310`,
  UNNEST(cpc) AS cpc
WHERE
  cpc.code LIKE "B64B%" OR cpc.code LIKE "B64C%" OR cpc.code LIKE "B64D%" OR cpc.code LIKE "B64F%" OR cpc.code LIKE "B64G%"
GROUP BY
  publication_number
HAVING
  COUNT(cpc.code) > 0""
a) UNNEST(cpc): Flattens the nested cpc array.
b) STRING_AGG(DISTINCT cpc.code, ' | '): Aggregates distinct CPC codes separated by |.
c) WHERE clause: Filters CPC codes by specified patterns.
d) HAVING COUNT(...) > 0: Ensures only patents with at least one valid CPC code are included.



3. assignee_data CTE:

Purpose: To collect and aggregate assignee country codes for each patent without filtering by specific country codes.
Details:
""SELECT
  publication_number,
  STRING_AGG(assignee_harmonized.country_code, ' | ') AS assignee_countries
FROM
  `patents-public-data.patents.publications_202310`,
  UNNEST(assignee_harmonized) AS assignee_harmonized
GROUP BY
  publication_number""
a) UNNEST(assignee_harmonized): Flattens the nested assignee_harmonized array.
b) STRING_AGG(assignee_harmonized.country_code, ' | '): Aggregates assignee country codes separated by |.



4. Main Query
Purpose: To join the results from the CTEs and include additional filters to generate the final dataset.
Details:
""SELECT
  p.publication_number,
  CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) AS publication_year,
  i.inventor_countries,
  c.cpc_codes,
  a.assignee_countries
FROM
  `patents-public-data.patents.publications_202310` AS p
JOIN
  inventor_data AS i
ON
  p.publication_number = i.publication_number
JOIN
  cpc_data AS c
ON
  p.publication_number = c.publication_number
LEFT JOIN
  assignee_data AS a
ON
  p.publication_number = a.publication_number
WHERE
  LEFT(p.publication_number, 2) = 'US'
  AND CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) BETWEEN 1989 AND 2008
LIMIT 100;""
a) SELECT Clause: Retrieves the publication number, publication year, inventor countries, CPC codes, and assignee countries.
b) JOIN Clauses: Joins the main table with the CTEs on publication_number:
b.1) INNER JOIN with inventor_data and cpc_data to ensure only patents with both valid inventor countries and CPC codes are included.
b.2) LEFT JOIN with assignee_data to include all patents, even those without assignee countries.
c) WHERE Clause: Filters to include only US patents and publication years between 1989 and 2008.
d) LIMIT Clause: Limits the results to 100 entries for simplicity.



5. Running the Query and Converting to DataFrame
Purpose: Execute the query, retrieve the results, convert them to a pandas DataFrame, and display the DataFrame.
Details:
a) client.query(query): Executes the SQL query.
b) query_job.result(): Retrieves the query results.
c) results.to_dataframe(): Converts the results to a pandas DataFrame.

In [ ]:
# SQL query to only retrieve data
query = """
WITH inventor_data AS (
  SELECT
    publication_number,
    STRING_AGG(inventor_harmonized.country_code, ' | ') AS inventor_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(inventor_harmonized) AS inventor_harmonized
  WHERE
    inventor_harmonized.country_code IN ("AT", "AU", "BE", "CA", "DK", "FI", "DE", "IE", "JP", "NL", "NO", "SE", "CH", "GB", "US")
  GROUP BY
    publication_number
  HAVING
    COUNT(inventor_harmonized.country_code) > 0
),
cpc_data AS (
  SELECT
    publication_number,
    STRING_AGG(DISTINCT cpc.code, ' | ') AS cpc_codes
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(cpc) AS cpc
  WHERE
    cpc.code LIKE "B64B%" OR cpc.code LIKE "B64C%" OR cpc.code LIKE "B64D%" OR cpc.code LIKE "B64F%" OR cpc.code LIKE "B64G%" OR
    cpc.code LIKE "A61K%" OR cpc.code LIKE "C07K%" OR cpc.code LIKE "A61P%" OR cpc.code LIKE "C12N%" OR cpc.code LIKE "C07D%" OR
    cpc.code LIKE "C12P%" OR cpc.code LIKE "C07H%" OR cpc.code LIKE "C12Q%" OR cpc.code LIKE "C07J%" OR cpc.code LIKE "B41J%" OR
    cpc.code LIKE "G05F%" OR cpc.code LIKE "G06G%" OR cpc.code LIKE "G06T%" OR cpc.code LIKE "G07G%" OR cpc.code LIKE "H03K%" OR
    cpc.code LIKE "B41K%" OR cpc.code LIKE "G06C%" OR cpc.code LIKE "G06J%" OR cpc.code LIKE "G07B%" OR cpc.code LIKE "G09D%" OR
    cpc.code LIKE "H03L%" OR cpc.code LIKE "B43M%" OR cpc.code LIKE "G06D%" OR cpc.code LIKE "G06K%" OR cpc.code LIKE "G07C%" OR
    cpc.code LIKE "G09G%" OR cpc.code LIKE "G02F%" OR cpc.code LIKE "G06E%" OR cpc.code LIKE "G06M%" OR cpc.code LIKE "G07D%" OR
    cpc.code LIKE "G10L%" OR cpc.code LIKE "G03G%" OR cpc.code LIKE "G06F%" OR cpc.code LIKE "G06N%" OR cpc.code LIKE "G07F%" OR
    cpc.code LIKE "G11B%" OR cpc.code LIKE "B81B%" OR cpc.code LIKE "G11C%" OR cpc.code LIKE "H01L%" OR cpc.code LIKE "H03B%" OR
    cpc.code LIKE "H03H%" OR cpc.code LIKE "H04J%" OR cpc.code LIKE "H04Q%" OR cpc.code LIKE "B81C%" OR cpc.code LIKE "H01C%" OR
    cpc.code LIKE "H01P%" OR cpc.code LIKE "H03C%" OR cpc.code LIKE "H03J%" OR cpc.code LIKE "H04K%" OR cpc.code LIKE "H04R%" OR
    cpc.code LIKE "G03H%" OR cpc.code LIKE "H01F%" OR cpc.code LIKE "H01Q%" OR cpc.code LIKE "H03D%" OR cpc.code LIKE "H03M%" OR
    cpc.code LIKE "H04L%" OR cpc.code LIKE "H04S%" OR cpc.code LIKE "G09B%" OR cpc.code LIKE "H01G%" OR cpc.code LIKE "H01S%" OR
    cpc.code LIKE "H03F%" OR cpc.code LIKE "H04B%" OR cpc.code LIKE "H04M%" OR cpc.code LIKE "H05K%" OR cpc.code LIKE "G09C%" OR
    cpc.code LIKE "H01J%" OR cpc.code LIKE "H02J%" OR cpc.code LIKE "H03G%" OR cpc.code LIKE "H04H%" OR cpc.code LIKE "H04N%" OR
    cpc.code LIKE "A61B%" OR cpc.code LIKE "A61J%" OR cpc.code LIKE "B04B%" OR cpc.code LIKE "G01F%" OR cpc.code LIKE "G01N%" OR
    cpc.code LIKE "G02C%" OR cpc.code LIKE "G04D%" OR cpc.code LIKE "G12B%" OR cpc.code LIKE "A61C%" OR cpc.code LIKE "A61L%" OR
    cpc.code LIKE "C12M%" OR cpc.code LIKE "G01H%" OR cpc.code LIKE "G01R%" OR cpc.code LIKE "G03B%" OR cpc.code LIKE "G04F%" OR
    cpc.code LIKE "G21G%" OR cpc.code LIKE "A61D%" OR cpc.code LIKE "A61M%" OR cpc.code LIKE "F15C%" OR cpc.code LIKE "G01J%" OR
    cpc.code LIKE "G01S%" OR cpc.code LIKE "G03D%" OR cpc.code LIKE "G04G%" OR cpc.code LIKE "G21K%" OR cpc.code LIKE "A61F%" OR
    cpc.code LIKE "A61N%" OR cpc.code LIKE "G01B%" OR cpc.code LIKE "G01K%" OR cpc.code LIKE "G01T%" OR cpc.code LIKE "G03F%" OR
    cpc.code LIKE "G05B%" OR cpc.code LIKE "H05G%" OR cpc.code LIKE "A61G%" OR cpc.code LIKE "A62B%" OR cpc.code LIKE "G01C%" OR
    cpc.code LIKE "G01L%" OR cpc.code LIKE "G01W%" OR cpc.code LIKE "G04B%" OR cpc.code LIKE "G08C%" OR cpc.code LIKE "A61H%" OR
    cpc.code LIKE "B01L%" OR cpc.code LIKE "G01D%" OR cpc.code LIKE "G01M%" OR cpc.code LIKE "G02B%" OR cpc.code LIKE "G04C%" OR
    cpc.code LIKE "G09F%" OR cpc.code LIKE "A01N%" OR cpc.code LIKE "C01D%" OR cpc.code LIKE "C06B%" OR cpc.code LIKE "C08F%" OR
    cpc.code LIKE "C09F%" OR cpc.code LIKE "C10K%" OR cpc.code LIKE "C25B%" OR cpc.code LIKE "G03C%" OR cpc.code LIKE "A62D%" OR
    cpc.code LIKE "C01F%" OR cpc.code LIKE "C06C%" OR cpc.code LIKE "C08G%" OR cpc.code LIKE "C09G%" OR cpc.code LIKE "C10M%" OR
    cpc.code LIKE "D01C%" OR cpc.code LIKE "G21F%" OR cpc.code LIKE "B01J%" OR cpc.code LIKE "C01G%" OR cpc.code LIKE "C06D%" OR
    cpc.code LIKE "C08H%" OR cpc.code LIKE "C09H%" OR cpc.code LIKE "C11B%" OR cpc.code LIKE "D01F%" OR cpc.code LIKE "B09B%" OR
    cpc.code LIKE "C02F%" OR cpc.code LIKE "C07B%" OR cpc.code LIKE "C08J%" OR cpc.code LIKE "C09J%" OR cpc.code LIKE "C11C%" OR
    cpc.code LIKE "D06L%" OR cpc.code LIKE "B09C%" OR cpc.code LIKE "C05B%" OR cpc.code LIKE "C07C%" OR cpc.code LIKE "C08K%" OR
    cpc.code LIKE "C09K%" OR cpc.code LIKE "C11D%" OR cpc.code LIKE "F17C%" OR cpc.code LIKE "B27K%" OR cpc.code LIKE "C05C%" OR
    cpc.code LIKE "C07F%" OR cpc.code LIKE "C08L%" OR cpc.code LIKE "C10B%" OR cpc.code LIKE "C12S%" OR cpc.code LIKE "F17D%" OR
    cpc.code LIKE "B29B%" OR cpc.code LIKE "C05D%" OR cpc.code LIKE "C07G%" OR cpc.code LIKE "C09B%" OR cpc.code LIKE "C10C%" OR
    cpc.code LIKE "C14C%" OR cpc.code LIKE "F25J%" OR cpc.code LIKE "C01B%" OR cpc.code LIKE "C05F%" OR cpc.code LIKE "C08B%" OR
        cpc.code LIKE "C08C%" OR cpc.code LIKE "C09D%" OR cpc.code LIKE "C10J%" OR cpc.code LIKE "C23G%" OR cpc.code LIKE "F42D%" OR
    cpc.code LIKE "B60M%" OR cpc.code LIKE "F21M%" OR cpc.code LIKE "G08B%" OR cpc.code LIKE "H01H%" OR cpc.code LIKE "H02B%" OR
    cpc.code LIKE "H02P%" OR cpc.code LIKE "B61L%" OR cpc.code LIKE "F21P%" OR cpc.code LIKE "G08G%" OR cpc.code LIKE "H01K%" OR
    cpc.code LIKE "H02H%" OR cpc.code LIKE "H05C%" OR cpc.code LIKE "F21H%" OR cpc.code LIKE "F21Q%" OR cpc.code LIKE "G10K%" OR
    cpc.code LIKE "H01M%" OR cpc.code LIKE "H02K%" OR cpc.code LIKE "F21K%" OR cpc.code LIKE "F21S%" OR cpc.code LIKE "G21C%" OR
    cpc.code LIKE "H01R%" OR cpc.code LIKE "H02M%" OR cpc.code LIKE "F21L%" OR cpc.code LIKE "F21V%" OR cpc.code LIKE "G21D%" OR
    cpc.code LIKE "H01T%" OR cpc.code LIKE "H02N%" OR cpc.code LIKE "A01B%" OR cpc.code LIKE "B01D%" OR cpc.code LIKE "B23D%" OR
    cpc.code LIKE "B31F%" OR cpc.code LIKE "B68F%" OR cpc.code LIKE "D06B%" OR cpc.code LIKE "F04B%" OR cpc.code LIKE "F23N%" OR
    cpc.code LIKE "F41J%" OR cpc.code LIKE "A01C%" OR cpc.code LIKE "B01F%" OR cpc.code LIKE "B23F%" OR cpc.code LIKE "B41B%" OR
    cpc.code LIKE "C10F%" OR cpc.code LIKE "D06F%" OR cpc.code LIKE "F04C%" OR cpc.code LIKE "F23R%" OR cpc.code LIKE "F42C%" OR
    cpc.code LIKE "A01D%" OR cpc.code LIKE "B02B%" OR cpc.code LIKE "B23G%" OR cpc.code LIKE "B41C%" OR cpc.code LIKE "C12L%" OR
    cpc.code LIKE "D06G%" OR cpc.code LIKE "F04D%" OR cpc.code LIKE "F24B%" OR cpc.code LIKE "G01G%" OR cpc.code LIKE "A01F%" OR
    cpc.code LIKE "B02C%" OR cpc.code LIKE "B23H%" OR cpc.code LIKE "B41D%" OR cpc.code LIKE "C13C%" OR cpc.code LIKE "D06H%" OR
    cpc.code LIKE "F04F%" OR cpc.code LIKE "F24C%" OR cpc.code LIKE "G21J%" OR cpc.code LIKE "A01G%" OR cpc.code LIKE "B03B%" OR
    cpc.code LIKE "B23K%" OR cpc.code LIKE "B41F%" OR cpc.code LIKE "C13D%" OR cpc.code LIKE "D21B%" OR cpc.code LIKE "F15B%" OR
    cpc.code LIKE "F24D%" OR cpc.code LIKE "H05B%" OR cpc.code LIKE "A01J%" OR cpc.code LIKE "B03C%" OR cpc.code LIKE "B23P%" OR
    cpc.code LIKE "B41G%" OR cpc.code LIKE "C13G%" OR cpc.code LIKE "D21D%" OR cpc.code LIKE "F16C%" OR cpc.code LIKE "F24F%" OR
    cpc.code LIKE "H05F%" OR cpc.code LIKE "A01K%" OR cpc.code LIKE "B03D%" OR cpc.code LIKE "B23Q%" OR cpc.code LIKE "B41L%" OR
    cpc.code LIKE "C13H%" OR cpc.code LIKE "D21F%" OR cpc.code LIKE "F16D%" OR cpc.code LIKE "F24H%" OR cpc.code LIKE "H05H%" OR
    cpc.code LIKE "A01M%" OR cpc.code LIKE "B04C%" OR cpc.code LIKE "B24B%" OR cpc.code LIKE "B41N%" OR cpc.code LIKE "C14B%" OR
    cpc.code LIKE "D21G%" OR cpc.code LIKE "F16F%" OR cpc.code LIKE "F25B%" OR cpc.code LIKE "A21B%" OR cpc.code LIKE "B05B%" OR
    cpc.code LIKE "B24C%" OR cpc.code LIKE "B42B%" OR cpc.code LIKE "C23C%" OR cpc.code LIKE "E01C%" OR cpc.code LIKE "F16G%" OR
    cpc.code LIKE "F25C%" OR cpc.code LIKE "A21C%" OR cpc.code LIKE "B05C%" OR cpc.code LIKE "B25D%" OR cpc.code LIKE "B42C%" OR
    cpc.code LIKE "D01B%" OR cpc.code LIKE "E02D%" OR cpc.code LIKE "F16H%" OR cpc.code LIKE "F25D%" OR cpc.code LIKE "A22B%" OR
    cpc.code LIKE "B05D%" OR cpc.code LIKE "B25J%" OR cpc.code LIKE "B44B%" OR cpc.code LIKE "D01D%" OR cpc.code LIKE "E02F%" OR
    cpc.code LIKE "F16K%" OR cpc.code LIKE "F26B%" OR cpc.code LIKE "A22C%" OR cpc.code LIKE "B06B%" OR cpc.code LIKE "B26D%" OR
    cpc.code LIKE "B61B%" OR cpc.code LIKE "D01G%" OR cpc.code LIKE "E06C%" OR cpc.code LIKE "F16M%" OR cpc.code LIKE "F27B%" OR
    cpc.code LIKE "A23N%" OR cpc.code LIKE "B07B%" OR cpc.code LIKE "B26F%" OR cpc.code LIKE "B63G%" OR cpc.code LIKE "D01H%" OR
    cpc.code LIKE "E21B%" OR cpc.code LIKE "F16N%" OR cpc.code LIKE "F28B%" OR cpc.code LIKE "A24C%" OR cpc.code LIKE "B07C%" OR
    cpc.code LIKE "B27B%" OR cpc.code LIKE "B65B%" OR cpc.code LIKE "D02G%" OR cpc.code LIKE "E21C%" OR cpc.code LIKE "F22D%" OR
    cpc.code LIKE "F28C%" OR cpc.code LIKE "A41H%" OR cpc.code LIKE "B08B%" OR cpc.code LIKE "B27C%" OR cpc.code LIKE "B65C%" OR
    cpc.code LIKE "D02H%" OR cpc.code LIKE "E21D%" OR cpc.code LIKE "F23B%" OR cpc.code LIKE "F28D%" OR cpc.code LIKE "A42C%" OR
    cpc.code LIKE "B21B%" OR cpc.code LIKE "B27F%" OR cpc.code LIKE "B65G%" OR cpc.code LIKE "D02J%" OR cpc.code LIKE "E21F%" OR
    cpc.code LIKE "F23C%" OR cpc.code LIKE "F28F%" OR cpc.code LIKE "A43D%" OR cpc.code LIKE "B21D%" OR cpc.code LIKE "B27J%" OR
    cpc.code LIKE "B65H%" OR cpc.code LIKE "D03C%" OR cpc.code LIKE "F01B%" OR cpc.code LIKE "F23D%" OR cpc.code LIKE "F28G%" OR
    cpc.code LIKE "A45D%" OR cpc.code LIKE "B21F%" OR cpc.code LIKE "B27L%" OR cpc.code LIKE "B66B%" OR cpc.code LIKE "D03D%" OR
    cpc.code LIKE "F01C%" OR cpc.code LIKE "F23G%" OR cpc.code LIKE "F41A%" OR cpc.code LIKE "A47G%" OR cpc.code LIKE "B21H%" OR
    cpc.code LIKE "B28D%" OR cpc.code LIKE "B66C%" OR cpc.code LIKE "D03J%" OR cpc.code LIKE "F01D%" OR cpc.code LIKE "F23H%" OR
    cpc.code LIKE "F41B%" OR cpc.code LIKE "A47J%" OR cpc.code LIKE "B21J%" OR cpc.code LIKE "B30B%" OR cpc.code LIKE "B66D%" OR
        cpc.code LIKE "D04B%" OR cpc.code LIKE "F03B%" OR cpc.code LIKE "F23J%" OR cpc.code LIKE "F41C%" OR cpc.code LIKE "A47L%" OR
    cpc.code LIKE "B22C%" OR cpc.code LIKE "B31B%" OR cpc.code LIKE "B66F%" OR cpc.code LIKE "D04C%" OR cpc.code LIKE "F03C%" OR
    cpc.code LIKE "F23K%" OR cpc.code LIKE "F41F%" OR cpc.code LIKE "A62C%" OR cpc.code LIKE "B23B%" OR cpc.code LIKE "B31C%" OR
    cpc.code LIKE "B67B%" OR cpc.code LIKE "D05B%" OR cpc.code LIKE "F03D%" OR cpc.code LIKE "F23L%" OR cpc.code LIKE "F41G%" OR
    cpc.code LIKE "B01B%" OR cpc.code LIKE "B23C%" OR cpc.code LIKE "B31D%" OR cpc.code LIKE "B67C%" OR cpc.code LIKE "D05C%" OR
    cpc.code LIKE "F03G%" OR cpc.code LIKE "F23M%" OR cpc.code LIKE "F41H%" OR cpc.code LIKE "B60B%" OR cpc.code LIKE "B60K%" OR
    cpc.code LIKE "B60R%" OR cpc.code LIKE "F01L%" OR cpc.code LIKE "F02D%" OR cpc.code LIKE "F02P%" OR cpc.code LIKE "B60D%" OR
    cpc.code LIKE "B60L%" OR cpc.code LIKE "B60S%" OR cpc.code LIKE "F01M%" OR cpc.code LIKE "F02F%" OR cpc.code LIKE "F16J%" OR
    cpc.code LIKE "B60G%" OR cpc.code LIKE "B60N%" OR cpc.code LIKE "B60T%" OR cpc.code LIKE "F01N%" OR cpc.code LIKE "F02G%" OR
    cpc.code LIKE "G01P%" OR cpc.code LIKE "B60H%" OR cpc.code LIKE "B60P%" OR cpc.code LIKE "B62D%" OR cpc.code LIKE "F01P%" OR
    cpc.code LIKE "F02M%" OR cpc.code LIKE "G05D%" OR cpc.code LIKE "B60J%" OR cpc.code LIKE "B60Q%" OR cpc.code LIKE "E01H%" OR
    cpc.code LIKE "F02B%" OR cpc.code LIKE "F02N%" OR cpc.code LIKE "G05G%" OR cpc.code LIKE "B61B%" OR cpc.code LIKE "B61H%" OR
    cpc.code LIKE "B62H%" OR cpc.code LIKE "B61C%" OR cpc.code LIKE "B61J%" OR cpc.code LIKE "B62J%" OR cpc.code LIKE "B61D%" OR
    cpc.code LIKE "B61K%" OR cpc.code LIKE "B62K%" OR cpc.code LIKE "B61F%" OR cpc.code LIKE "B61L%" OR cpc.code LIKE "B62M%" OR
    cpc.code LIKE "B61G%" OR cpc.code LIKE "B62B%" OR cpc.code LIKE "B65G%"
GROUP BY
    publication_number
  HAVING
    COUNT(cpc.code) > 0
),
assignee_data AS (
  SELECT
    publication_number,
    STRING_AGG(assignee_harmonized.country_code, ' | ') AS assignee_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(assignee_harmonized) AS assignee_harmonized
  GROUP BY
    publication_number
)
SELECT
  p.publication_number,
  CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) AS publication_year,
  i.inventor_countries,
  c.cpc_codes,
  a.assignee_countries
FROM
  `patents-public-data.patents.publications_202310` AS p
JOIN
  inventor_data AS i
ON
  p.publication_number = i.publication_number
JOIN
  cpc_data AS c
ON
  p.publication_number = c.publication_number
LEFT JOIN
  assignee_data AS a
ON
  p.publication_number = a.publication_number
WHERE
  LEFT(p.publication_number, 2) = 'US'
  AND CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) BETWEEN 1989 AND 2008
LIMIT 1000;
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert to DataFrame
df = results.to_dataframe()

# Display the DataFrame
df




,publication_number,publication_year,inventor_countries,cpc_codes,assignee_countries
0,US-2002157479-A1,2002,US | US,G01F1/8477 | G01F1/8409 | G01F1/84 | G01F1/8413,None
1,US-2002045186-A1,2002,NL | NL | NL,C07K14/47 | A61K38/00,None
2,US-2002107164-A1,2002,US | US,C11D1/123 | C11D1/143 | C11D1/90 | C11D1/29 | ...,US
3,US-2002194853-A1,2002,US,F25D3/102 | F25D29/001,None
4,US-2002053669-A1,2002,JP | JP,H01L29/66765 | H01L27/1288 | G02F1/13454 | H01...,US
...,...,...,...,...,...
995,US-2008212473-A1,2008,US | US | US | US,H04L12/66,DE
996,US-2008228736-A1,2008,US | US | US | US,H04M7/0036 | H04M3/42068 | H04M3/2218 | H04M22...,US | US
997,US-2005233221-A1,2005,JP | JP,H01M8/10 | H01M8/1023 | H01M10/0565 | H01G11/5...,
998,US-2004207802-A1,2004,US,G02B27/145 | H04N13/239 | H04N13/398 | H04N13/...,None


Check if the data are retrieved correctly

In [ ]:
# SQL query to look at the names and countries of inventors for the patent with publication_number "US-5928469-A"
query = """
SELECT
  publication_number,
  inventor_harmonized.name AS inventor_name,
  inventor_harmonized.country_code AS inventor_country
FROM
  `patents-public-data.patents.publications_202310`,
  UNNEST(inventor_harmonized) AS inventor_harmonized
WHERE
  publication_number = 'US-2002075467-A1';
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert to DataFrame
df2 = results.to_dataframe()

# Display the DataFrame
df2

,publication_number,inventor_name,inventor_country
0,US-2002075467-A1,TANAKA KEIICHI,JP
1,US-2002075467-A1,BINNARD MIKE,US


Retrive the actual data without limiting the retreival to 1000

In [ ]:
# SQL query to only retrieve data
query = """
WITH inventor_data AS (
  SELECT
    publication_number,
    STRING_AGG(inventor_harmonized.country_code, ' | ') AS inventor_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(inventor_harmonized) AS inventor_harmonized
  WHERE
    inventor_harmonized.country_code IN ("AT", "AU", "BE", "CA", "DK", "FI", "DE", "IE", "JP", "NL", "NO", "SE", "CH", "GB", "US")
  GROUP BY
    publication_number
  HAVING
    COUNT(inventor_harmonized.country_code) > 0
),
cpc_data AS (
  SELECT
    publication_number,
    STRING_AGG(DISTINCT cpc.code, ' | ') AS cpc_codes
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(cpc) AS cpc
  WHERE
    cpc.code LIKE "B64B%" OR cpc.code LIKE "B64C%" OR cpc.code LIKE "B64D%" OR cpc.code LIKE "B64F%" OR cpc.code LIKE "B64G%" OR
    cpc.code LIKE "A61K%" OR cpc.code LIKE "C07K%" OR cpc.code LIKE "A61P%" OR cpc.code LIKE "C12N%" OR cpc.code LIKE "C07D%" OR
    cpc.code LIKE "C12P%" OR cpc.code LIKE "C07H%" OR cpc.code LIKE "C12Q%" OR cpc.code LIKE "C07J%" OR cpc.code LIKE "B41J%" OR
    cpc.code LIKE "G05F%" OR cpc.code LIKE "G06G%" OR cpc.code LIKE "G06T%" OR cpc.code LIKE "G07G%" OR cpc.code LIKE "H03K%" OR
    cpc.code LIKE "B41K%" OR cpc.code LIKE "G06C%" OR cpc.code LIKE "G06J%" OR cpc.code LIKE "G07B%" OR cpc.code LIKE "G09D%" OR
    cpc.code LIKE "H03L%" OR cpc.code LIKE "B43M%" OR cpc.code LIKE "G06D%" OR cpc.code LIKE "G06K%" OR cpc.code LIKE "G07C%" OR
    cpc.code LIKE "G09G%" OR cpc.code LIKE "G02F%" OR cpc.code LIKE "G06E%" OR cpc.code LIKE "G06M%" OR cpc.code LIKE "G07D%" OR
    cpc.code LIKE "G10L%" OR cpc.code LIKE "G03G%" OR cpc.code LIKE "G06F%" OR cpc.code LIKE "G06N%" OR cpc.code LIKE "G07F%" OR
    cpc.code LIKE "G11B%" OR cpc.code LIKE "B81B%" OR cpc.code LIKE "G11C%" OR cpc.code LIKE "H01L%" OR cpc.code LIKE "H03B%" OR
    cpc.code LIKE "H03H%" OR cpc.code LIKE "H04J%" OR cpc.code LIKE "H04Q%" OR cpc.code LIKE "B81C%" OR cpc.code LIKE "H01C%" OR
    cpc.code LIKE "H01P%" OR cpc.code LIKE "H03C%" OR cpc.code LIKE "H03J%" OR cpc.code LIKE "H04K%" OR cpc.code LIKE "H04R%" OR
    cpc.code LIKE "G03H%" OR cpc.code LIKE "H01F%" OR cpc.code LIKE "H01Q%" OR cpc.code LIKE "H03D%" OR cpc.code LIKE "H03M%" OR
    cpc.code LIKE "H04L%" OR cpc.code LIKE "H04S%" OR cpc.code LIKE "G09B%" OR cpc.code LIKE "H01G%" OR cpc.code LIKE "H01S%" OR
    cpc.code LIKE "H03F%" OR cpc.code LIKE "H04B%" OR cpc.code LIKE "H04M%" OR cpc.code LIKE "H05K%" OR cpc.code LIKE "G09C%" OR
    cpc.code LIKE "H01J%" OR cpc.code LIKE "H02J%" OR cpc.code LIKE "H03G%" OR cpc.code LIKE "H04H%" OR cpc.code LIKE "H04N%" OR
    cpc.code LIKE "A61B%" OR cpc.code LIKE "A61J%" OR cpc.code LIKE "B04B%" OR cpc.code LIKE "G01F%" OR cpc.code LIKE "G01N%" OR
    cpc.code LIKE "G02C%" OR cpc.code LIKE "G04D%" OR cpc.code LIKE "G12B%" OR cpc.code LIKE "A61C%" OR cpc.code LIKE "A61L%" OR
    cpc.code LIKE "C12M%" OR cpc.code LIKE "G01H%" OR cpc.code LIKE "G01R%" OR cpc.code LIKE "G03B%" OR cpc.code LIKE "G04F%" OR
    cpc.code LIKE "G21G%" OR cpc.code LIKE "A61D%" OR cpc.code LIKE "A61M%" OR cpc.code LIKE "F15C%" OR cpc.code LIKE "G01J%" OR
    cpc.code LIKE "G01S%" OR cpc.code LIKE "G03D%" OR cpc.code LIKE "G04G%" OR cpc.code LIKE "G21K%" OR cpc.code LIKE "A61F%" OR
    cpc.code LIKE "A61N%" OR cpc.code LIKE "G01B%" OR cpc.code LIKE "G01K%" OR cpc.code LIKE "G01T%" OR cpc.code LIKE "G03F%" OR
    cpc.code LIKE "G05B%" OR cpc.code LIKE "H05G%" OR cpc.code LIKE "A61G%" OR cpc.code LIKE "A62B%" OR cpc.code LIKE "G01C%" OR
    cpc.code LIKE "G01L%" OR cpc.code LIKE "G01W%" OR cpc.code LIKE "G04B%" OR cpc.code LIKE "G08C%" OR cpc.code LIKE "A61H%" OR
    cpc.code LIKE "B01L%" OR cpc.code LIKE "G01D%" OR cpc.code LIKE "G01M%" OR cpc.code LIKE "G02B%" OR cpc.code LIKE "G04C%" OR
    cpc.code LIKE "G09F%" OR cpc.code LIKE "A01N%" OR cpc.code LIKE "C01D%" OR cpc.code LIKE "C06B%" OR cpc.code LIKE "C08F%" OR
    cpc.code LIKE "C09F%" OR cpc.code LIKE "C10K%" OR cpc.code LIKE "C25B%" OR cpc.code LIKE "G03C%" OR cpc.code LIKE "A62D%" OR
    cpc.code LIKE "C01F%" OR cpc.code LIKE "C06C%" OR cpc.code LIKE "C08G%" OR cpc.code LIKE "C09G%" OR cpc.code LIKE "C10M%" OR
    cpc.code LIKE "D01C%" OR cpc.code LIKE "G21F%" OR cpc.code LIKE "B01J%" OR cpc.code LIKE "C01G%" OR cpc.code LIKE "C06D%" OR
    cpc.code LIKE "C08H%" OR cpc.code LIKE "C09H%" OR cpc.code LIKE "C11B%" OR cpc.code LIKE "D01F%" OR cpc.code LIKE "B09B%" OR
    cpc.code LIKE "C02F%" OR cpc.code LIKE "C07B%" OR cpc.code LIKE "C08J%" OR cpc.code LIKE "C09J%" OR cpc.code LIKE "C11C%" OR
    cpc.code LIKE "D06L%" OR cpc.code LIKE "B09C%" OR cpc.code LIKE "C05B%" OR cpc.code LIKE "C07C%" OR cpc.code LIKE "C08K%" OR
    cpc.code LIKE "C09K%" OR cpc.code LIKE "C11D%" OR cpc.code LIKE "F17C%" OR cpc.code LIKE "B27K%" OR cpc.code LIKE "C05C%" OR
    cpc.code LIKE "C07F%" OR cpc.code LIKE "C08L%" OR cpc.code LIKE "C10B%" OR cpc.code LIKE "C12S%" OR cpc.code LIKE "F17D%" OR
    cpc.code LIKE "B29B%" OR cpc.code LIKE "C05D%" OR cpc.code LIKE "C07G%" OR cpc.code LIKE "C09B%" OR cpc.code LIKE "C10C%" OR
    cpc.code LIKE "C14C%" OR cpc.code LIKE "F25J%" OR cpc.code LIKE "C01B%" OR cpc.code LIKE "C05F%" OR cpc.code LIKE "C08B%" OR
    cpc.code LIKE "C08C%" OR cpc.code LIKE "C09D%" OR cpc.code LIKE "C10J%" OR cpc.code LIKE "C23G%" OR cpc.code LIKE "F42D%" OR
    cpc.code LIKE "B60M%" OR cpc.code LIKE "F21M%" OR cpc.code LIKE "G08B%" OR cpc.code LIKE "H01H%" OR cpc.code LIKE "H02B%" OR
    cpc.code LIKE "H02P%" OR cpc.code LIKE "B61L%" OR cpc.code LIKE "F21P%" OR cpc.code LIKE "G08G%" OR cpc.code LIKE "H01K%" OR
    cpc.code LIKE "H02H%" OR cpc.code LIKE "H05C%" OR cpc.code LIKE "F21H%" OR cpc.code LIKE "F21Q%" OR cpc.code LIKE "G10K%" OR
    cpc.code LIKE "H01M%" OR cpc.code LIKE "H02K%" OR cpc.code LIKE "F21K%" OR cpc.code LIKE "F21S%" OR cpc.code LIKE "G21C%" OR
    cpc.code LIKE "H01R%" OR cpc.code LIKE "H02M%" OR cpc.code LIKE "F21L%" OR cpc.code LIKE "F21V%" OR cpc.code LIKE "G21D%" OR
    cpc.code LIKE "H01T%" OR cpc.code LIKE "H02N%" OR cpc.code LIKE "A01B%" OR cpc.code LIKE "B01D%" OR cpc.code LIKE "B23D%" OR
    cpc.code LIKE "B31F%" OR cpc.code LIKE "B68F%" OR cpc.code LIKE "D06B%" OR cpc.code LIKE "F04B%" OR cpc.code LIKE "F23N%" OR
    cpc.code LIKE "F41J%" OR cpc.code LIKE "A01C%" OR cpc.code LIKE "B01F%" OR cpc.code LIKE "B23F%" OR cpc.code LIKE "B41B%" OR
    cpc.code LIKE "C10F%" OR cpc.code LIKE "D06F%" OR cpc.code LIKE "F04C%" OR cpc.code LIKE "F23R%" OR cpc.code LIKE "F42C%" OR
    cpc.code LIKE "A01D%" OR cpc.code LIKE "B02B%" OR cpc.code LIKE "B23G%" OR cpc.code LIKE "B41C%" OR cpc.code LIKE "C12L%" OR
    cpc.code LIKE "D06G%" OR cpc.code LIKE "F04D%" OR cpc.code LIKE "F24B%" OR cpc.code LIKE "G01G%" OR cpc.code LIKE "A01F%" OR
    cpc.code LIKE "B02C%" OR cpc.code LIKE "B23H%" OR cpc.code LIKE "B41D%" OR cpc.code LIKE "C13C%" OR cpc.code LIKE "D06H%" OR
    cpc.code LIKE "F04F%" OR cpc.code LIKE "F24C%" OR cpc.code LIKE "G21J%" OR cpc.code LIKE "A01G%" OR cpc.code LIKE "B03B%" OR
    cpc.code LIKE "B23K%" OR cpc.code LIKE "B41F%" OR cpc.code LIKE "C13D%" OR cpc.code LIKE "D21B%" OR cpc.code LIKE "F15B%" OR
    cpc.code LIKE "F24D%" OR cpc.code LIKE "H05B%" OR cpc.code LIKE "A01J%" OR cpc.code LIKE "B03C%" OR cpc.code LIKE "B23P%" OR
    cpc.code LIKE "B41G%" OR cpc.code LIKE "C13G%" OR cpc.code LIKE "D21D%" OR cpc.code LIKE "F16C%" OR cpc.code LIKE "F24F%" OR
    cpc.code LIKE "H05F%" OR cpc.code LIKE "A01K%" OR cpc.code LIKE "B03D%" OR cpc.code LIKE "B23Q%" OR cpc.code LIKE "B41L%" OR
    cpc.code LIKE "C13H%" OR cpc.code LIKE "D21F%" OR cpc.code LIKE "F16D%" OR cpc.code LIKE "F24H%" OR cpc.code LIKE "H05H%" OR
    cpc.code LIKE "A01M%" OR cpc.code LIKE "B04C%" OR cpc.code LIKE "B24B%" OR cpc.code LIKE "B41N%" OR cpc.code LIKE "C14B%" OR
    cpc.code LIKE "D21G%" OR cpc.code LIKE "F16F%" OR cpc.code LIKE "F25B%" OR cpc.code LIKE "A21B%" OR cpc.code LIKE "B05B%" OR
    cpc.code LIKE "B24C%" OR cpc.code LIKE "B42B%" OR cpc.code LIKE "C23C%" OR cpc.code LIKE "E01C%" OR cpc.code LIKE "F16G%" OR
    cpc.code LIKE "F25C%" OR cpc.code LIKE "A21C%" OR cpc.code LIKE "B05C%" OR cpc.code LIKE "B25D%" OR cpc.code LIKE "B42C%" OR
    cpc.code LIKE "D01B%" OR cpc.code LIKE "E02D%" OR cpc.code LIKE "F16H%" OR cpc.code LIKE "F25D%" OR cpc.code LIKE "A22B%" OR
    cpc.code LIKE "B05D%" OR cpc.code LIKE "B25J%" OR cpc.code LIKE "B44B%" OR cpc.code LIKE "D01D%" OR cpc.code LIKE "E02F%" OR
    cpc.code LIKE "F16K%" OR cpc.code LIKE "F26B%" OR cpc.code LIKE "A22C%" OR cpc.code LIKE "B06B%" OR cpc.code LIKE "B26D%" OR
    cpc.code LIKE "B61B%" OR cpc.code LIKE "D01G%" OR cpc.code LIKE "E06C%" OR cpc.code LIKE "F16M%" OR cpc.code LIKE "F27B%" OR
    cpc.code LIKE "A23N%" OR cpc.code LIKE "B07B%" OR cpc.code LIKE "B26F%" OR cpc.code LIKE "B63G%" OR cpc.code LIKE "D01H%" OR
    cpc.code LIKE "E21B%" OR cpc.code LIKE "F16N%" OR cpc.code LIKE "F28B%" OR cpc.code LIKE "A24C%" OR cpc.code LIKE "B07C%" OR
    cpc.code LIKE "B27B%" OR cpc.code LIKE "B65B%" OR cpc.code LIKE "D02G%" OR cpc.code LIKE "E21C%" OR cpc.code LIKE "F22D%" OR
    cpc.code LIKE "F28C%" OR cpc.code LIKE "A41H%" OR cpc.code LIKE "B08B%" OR cpc.code LIKE "B27C%" OR cpc.code LIKE "B65C%" OR
    cpc.code LIKE "D02H%" OR cpc.code LIKE "E21D%" OR cpc.code LIKE "F23B%" OR cpc.code LIKE "F28D%" OR cpc.code LIKE "A42C%" OR
    cpc.code LIKE "B21B%" OR cpc.code LIKE "B27F%" OR cpc.code LIKE "B65G%" OR cpc.code LIKE "D02J%" OR cpc.code LIKE "E21F%" OR
    cpc.code LIKE "F23C%" OR cpc.code LIKE "F28F%" OR cpc.code LIKE "A43D%" OR cpc.code LIKE "B21D%" OR cpc.code LIKE "B27J%" OR
    cpc.code LIKE "B65H%" OR cpc.code LIKE "D03C%" OR cpc.code LIKE "F01B%" OR cpc.code LIKE "F23D%" OR cpc.code LIKE "F28G%" OR
    cpc.code LIKE "A45D%" OR cpc.code LIKE "B21F%" OR cpc.code LIKE "B27L%" OR cpc.code LIKE "B66B%" OR cpc.code LIKE "D03D%" OR
    cpc.code LIKE "F01C%" OR cpc.code LIKE "F23G%" OR cpc.code LIKE "F41A%" OR cpc.code LIKE "A47G%" OR cpc.code LIKE "B21H%" OR
    cpc.code LIKE "B28D%" OR cpc.code LIKE "B66C%" OR cpc.code LIKE "D03J%" OR cpc.code LIKE "F01D%" OR cpc.code LIKE "F23H%" OR
    cpc.code LIKE "F41B%" OR cpc.code LIKE "A47J%" OR cpc.code LIKE "B21J%" OR cpc.code LIKE "B30B%" OR cpc.code LIKE "B66D%" OR
    cpc.code LIKE "D04B%" OR cpc.code LIKE "F03B%" OR cpc.code LIKE "F23J%" OR cpc.code LIKE "F41C%" OR cpc.code LIKE "A47L%" OR
    cpc.code LIKE "B22C%" OR cpc.code LIKE "B31B%" OR cpc.code LIKE "B66F%" OR cpc.code LIKE "D04C%" OR cpc.code LIKE "F03C%" OR
    cpc.code LIKE "F23K%" OR cpc.code LIKE "F41F%" OR cpc.code LIKE "A62C%" OR cpc.code LIKE "B23B%" OR cpc.code LIKE "B31C%" OR
    cpc.code LIKE "B67B%" OR cpc.code LIKE "D05B%" OR cpc.code LIKE "F03D%" OR cpc.code LIKE "F23L%" OR cpc.code LIKE "F41G%" OR
    cpc.code LIKE "B01B%" OR cpc.code LIKE "B23C%" OR cpc.code LIKE "B31D%" OR cpc.code LIKE "B67C%" OR cpc.code LIKE "D05C%" OR
    cpc.code LIKE "F03G%" OR cpc.code LIKE "F23M%" OR cpc.code LIKE "F41H%" OR cpc.code LIKE "B60B%" OR cpc.code LIKE "B60K%" OR
    cpc.code LIKE "B60R%" OR cpc.code LIKE "F01L%" OR cpc.code LIKE "F02D%" OR cpc.code LIKE "F02P%" OR cpc.code LIKE "B60D%" OR
    cpc.code LIKE "B60L%" OR cpc.code LIKE "B60S%" OR cpc.code LIKE "F01M%" OR cpc.code LIKE "F02F%" OR cpc.code LIKE "F16J%" OR
    cpc.code LIKE "B60G%" OR cpc.code LIKE "B60N%" OR cpc.code LIKE "B60T%" OR cpc.code LIKE "F01N%" OR cpc.code LIKE "F02G%" OR
    cpc.code LIKE "G01P%" OR cpc.code LIKE "B60H%" OR cpc.code LIKE "B60P%" OR cpc.code LIKE "B62D%" OR cpc.code LIKE "F01P%" OR
    cpc.code LIKE "F02M%" OR cpc.code LIKE "G05D%" OR cpc.code LIKE "B60J%" OR cpc.code LIKE "B60Q%" OR cpc.code LIKE "E01H%" OR
    cpc.code LIKE "F02B%" OR cpc.code LIKE "F02N%" OR cpc.code LIKE "G05G%" OR cpc.code LIKE "B61B%" OR cpc.code LIKE "B61H%" OR
    cpc.code LIKE "B62H%" OR cpc.code LIKE "B61C%" OR cpc.code LIKE "B61J%" OR cpc.code LIKE "B62J%" OR cpc.code LIKE "B61D%" OR
    cpc.code LIKE "B61K%" OR cpc.code LIKE "B62K%" OR cpc.code LIKE "B61F%" OR cpc.code LIKE "B61L%" OR cpc.code LIKE "B62M%" OR
    cpc.code LIKE "B61G%" OR cpc.code LIKE "B62B%" OR cpc.code LIKE "B65G%"
GROUP BY
    publication_number
  HAVING
    COUNT(cpc.code) > 0
),
assignee_data AS (
  SELECT
    publication_number,
    STRING_AGG(assignee_harmonized.country_code, ' | ') AS assignee_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(assignee_harmonized) AS assignee_harmonized
  GROUP BY
    publication_number
)
SELECT
  p.publication_number,
  CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) AS publication_year,
  i.inventor_countries,
  c.cpc_codes,
  a.assignee_countries
FROM
  `patents-public-data.patents.publications_202310` AS p
JOIN
  inventor_data AS i
ON
  p.publication_number = i.publication_number
JOIN
  cpc_data AS c
ON
  p.publication_number = c.publication_number
LEFT JOIN
  assignee_data AS a
ON
  p.publication_number = a.publication_number
WHERE
  LEFT(p.publication_number, 2) = 'US'
  AND CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) BETWEEN 1989 AND 2008;
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert to DataFrame
df = results.to_dataframe()

# Display the DataFrame
df


,publication_number,publication_year,inventor_countries,cpc_codes,assignee_countries
0,US-2002035033-A1,2002,DE | DE | DE,B01J35/008 | B01J37/0234 | B01J35/0046 | B01J3...,None
1,US-2002013080-A1,2002,US | US | US,H01R13/521 | H01R9/226 | H01R2201/26 | B60R16/...,None
2,US-2002050888-A1,2002,JP | JP,H01H71/2472 | H01H71/1054 | H01H71/522,None
3,US-2001039675-A1,2001,US,A47G7/06,None
4,US-2002011400-A1,2002,US | US | US,B65G15/62 | B65G21/2081 | B65G2201/04,None
...,...,...,...,...,...
3617575,US-2007155195-A1,2007,US,H01R13/6477 | H01R13/6461,| |
3617576,US-2007225525-A1,2007,CH,C07C213/08 | C07C213/00 | A61P25/06 | C07C213/...,US
3617577,US-2007189481-A1,2007,US | US | US | US,H04M15/06,US
3617578,US-2006173666-A1,2006,US | US | US | US | US,G06F30/3323,


Save the retrieved data into a csv file

In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregate_patents_full.csv'
df.to_csv(output_file_path, index=False)

In [ ]:
# Function to handle None values during aggregation
def join_with_pipe(x):
    return ' | '.join([str(i) for i in x if i is not None])

# Grouping and aggregating the DataFrame, including assignee_countries
grouped_df = df.groupby(['publication_year', 'inventor_countries', 'publication_number']).agg({
    'cpc_codes': join_with_pipe,
    'assignee_countries': join_with_pipe  # Join assignee_countries with ' | ' separator
}).reset_index()

# Rename the columns
grouped_df.rename(columns={
    'publication_number': 'publication_number',
    'inventor_countries': 'inventor_country',
    'assignee_countries': 'assignee_country',
    'cpc_codes': 'patents_cpcs'
}, inplace=True)

grouped_df

,publication_year,inventor_country,publication_number,patents_cpcs,assignee_country
0,1989,AT,US-4794964-A,B27B7/02 | B27B29/00 | B27B5/187 | B23D45/102 ...,AT
1,1989,AT,US-4795474-A,A61F2002/5067 | A61F2/64 | A61F2002/5075 | F16...,DE
2,1989,AT,US-4795646-A,A61K31/17 | A61K9/1688 | A61P9/00,AT
3,1989,AT,US-4795750-A,A61P9/12 | C07D239/95 | C07D403/12 | C07D401/14,AT
4,1989,AT,US-4795967-A,G01F23/266 | G01F23/268,AT
...,...,...,...,...,...
3617575,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201847-A1,A61G7/05715 | A61G7/018 | A61B5/6891 | A61G7/0...,| | | | | | | | | | | | | | | ...
3617576,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201851-A1,A61G7/005 | A61G2203/42 | A61G7/0514 | A61G220...,| | | | | | | | | | | | | | | ...
3617577,2008,US | US | US | US | US | US | US | US | US | U...,US-2008289108-A1,A61G7/0506 | A61G7/05715 | A61G7/018 | A61G7/0...,| | | | | | | | | | | | | | | ...
3617578,2008,US | US | US | US | US | US | US | US | US | U...,US-7406731-B2,A61G7/0527 | A61G7/05769 | A61G7/0507 | A61G7/...,US


In [ ]:
# Display the number of rows in the original and grouped DataFrames
print(f"Original DataFrame rows: {df.shape[0]}")
print(f"Grouped DataFrame rows: {grouped_df.shape[0]}")

Original DataFrame rows: 3617580
Grouped DataFrame rows: 3617580


Save the dataset

In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregate_patents_cpcs_with_number_in_one_row.csv'
grouped_df.to_csv(output_file_path, index=False)

Remove the numbers after the first four characters of the "patents_cpcs".


Step 1: Define a Function to Simplify the CPC Codes

1. Function Definition:

def simplify_cpc_codes(codes): Defines a function named simplify_cpc_codes that takes a single argument codes.

2. Initialize List:

simplified_codes = []: Initializes an empty list to store the simplified codes.

3. Split Codes:

for code in codes.split('|'): Iterates over each code in the codes string, splitting the string by the | separator.

4. Regular Expression Match:

simplified_code = re.match(r'^[A-Z]\d{2}[A-Z]', code): Uses a regular expression to match the pattern where a code starts with an uppercase letter, followed by two digits, and another uppercase letter. This is intended to capture a simplified form of the CPC code.

5. Conditional Append:

if simplified_code: Checks if the regular expression found a match.
simplified_codes.append(simplified_code.group(0)): If a match is found, appends the simplified code to the simplified_codes list.
else: simplified_codes.append(code): If no match is found, appends the original code to the simplified_codes list.

6. Join Simplified Codes:

return '|'.join(simplified_codes): Joins the simplified codes with a | separator and returns the resulting string.



Step 2: Define a Function to Simplify Codes to Prefixes

1. Function Definition:

def simplify_to_prefix(codes): Defines a function named simplify_to_prefix that takes a single argument codes.

2. Simplify Codes to Prefixes:

simplified_codes = [code[:4] for code in codes.split('|')]: Uses a list comprehension to iterate over each code in the codes string, splitting the string by the | separator, and then taking only the first four characters of each code.

3. Join Simplified Codes:

return '|'.join(simplified_codes): Joins the simplified codes with a | separator and returns the resulting string.


Step 3: Apply the Function to the 'patents_cpcs' Column

1. Apply Function:

df['patents_cpcs'] = df['patents_cpcs'].apply(simplify_to_prefix): Applies the simplify_to_prefix function to each value in the patents_cpcs column of the DataFrame df.

2. Print DataFrame:

print(df.head()): Prints the first few rows of the DataFrame df to verify the changes.

In [ ]:
# Define a function to simplify the cpc codes
def simplify_cpc_codes(codes):
    simplified_codes = []
    for code in codes.split('|'):
        simplified_code = re.match(r'^[A-Z]\d{2}[A-Z]', code)
        if simplified_code:
            simplified_codes.append(simplified_code.group(0))
        else:
            simplified_codes.append(code)
    return '|'.join(simplified_codes)

# Adjust the function to remove characters after the first four and ensure at least 4 characters long
def simplify_to_prefix(codes):
    simplified_codes = []
    for code in codes.split('|'):
        code = code.strip()
        if len(code) < 4:
            simplified_code = code.ljust(4)  # Pad with spaces if less than 4 characters
        else:
            simplified_code = code[:4]  # Take only the first 4 characters
        simplified_codes.append(simplified_code)
    return '|'.join(simplified_codes)

# Apply the function to the 'patents_cpcs' column
grouped_df['patents_cpcs'] = grouped_df['patents_cpcs'].apply(simplify_to_prefix)
grouped_df
df = grouped_df

In [ ]:
# Display the number of rows in the original and grouped DataFrames
print(f"Original DataFrame rows: {df.shape[0]}")

Original DataFrame rows: 3617580


In [ ]:
df

,publication_year,inventor_country,publication_number,patents_cpcs,assignee_country
0,1989,AT,US-4794964-A,B27B|B27B|B27B|B23D|B27B,AT
1,1989,AT,US-4795474-A,A61F|A61F|A61F|F16C|A61F,DE
2,1989,AT,US-4795646-A,A61K|A61K|A61P,AT
3,1989,AT,US-4795750-A,A61P|C07D|C07D|C07D,AT
4,1989,AT,US-4795967-A,G01F|G01F,AT
...,...,...,...,...,...
3617575,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201847-A1,A61G|A61G|A61B|A61G|A61G|A61G|A61G|A61G|A61G|A...,| | | | | | | | | | | | | | | ...
3617576,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201851-A1,A61G|A61G|A61G|A61G|G01G|A61B|A61G|A61G|A61G|A...,| | | | | | | | | | | | | | | ...
3617577,2008,US | US | US | US | US | US | US | US | US | U...,US-2008289108-A1,A61G|A61G|A61G|A61G|A61G|A61G|A61G|A61G|G01G|A...,| | | | | | | | | | | | | | | ...
3617578,2008,US | US | US | US | US | US | US | US | US | U...,US-7406731-B2,A61G|A61G|A61G|A61G|A61G|A61G|G01G|A61G|A61G|A...,US


Sorting the DataFrame by 'publication_year' and 'inventor_country'

1. sort_values Method:

df.sort_values(by=['publication_year', 'inventor_country'])
This method is used to sort the DataFrame df based on the values in the specified columns.

2. by Parameter:

by=['publication_year', 'inventor_country']
This specifies the columns to sort by.
The DataFrame will first be sorted by the values in the publication_year column.
Within each publication_year, the DataFrame will be further sorted by the values in the inventor_country column.

3. Return Value:

The sort_values method returns a new DataFrame that is sorted according to the specified columns.
The sorted DataFrame is assigned to the variable df_sorted.


The purpose of this code is to sort the DataFrame df in an orderly manner based on the values in the publication_year and inventor_country columns. Sorting the DataFrame can make it easier to analyze trends and patterns in the data, especially when dealing with time series data or when trying to compare records within specific categories.

In [ ]:
df_sorted = df.sort_values(by=['publication_year', 'inventor_country'])

# Display the sorted DataFrame
print(df_sorted.head())
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregate_patents_cpcs_final.csv'
df_sorted.to_csv(output_file_path, index=False)

   publication_year inventor_country publication_number  \
0              1989               AT       US-4794964-A   
1              1989               AT       US-4795474-A   
2              1989               AT       US-4795646-A   
3              1989               AT       US-4795750-A   
4              1989               AT       US-4795967-A   

               patents_cpcs assignee_country  
0  B27B|B27B|B27B|B23D|B27B               AT  
1  A61F|A61F|A61F|F16C|A61F               DE  
2            A61K|A61K|A61P               AT  
3       A61P|C07D|C07D|C07D               AT  
4                 G01F|G01F               AT  


In [ ]:
# Display the number of rows in the original and grouped DataFrames
print(f"Original DataFrame rows: {df_sorted.shape[0]}")

Original DataFrame rows: 3617580


Check if there are multiple codes for each country in the same year, to make sure that the codes are properly executed

In [ ]:
# Filter the DataFrame for the US in the year 2005
filtered_df = df_sorted[(df_sorted['publication_year'] == 2005) & (df_sorted['inventor_country'] == 'US')]

# Count the occurrences of "B65B" in the 'patents_cpcs' column
count_a61p = filtered_df['patents_cpcs'].str.split('|').apply(lambda x: x.count('A61P')).sum()

count_a61p

7170

Load the datasets here: Optional

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregate_patents_cpcs_final.csv')
df


,publication_year,inventor_country,publication_number,patents_cpcs,assignee_country
0,1989,AT,US-4794964-A,B27B|B27B|B27B|B23D|B27B,AT
1,1989,AT,US-4795474-A,A61F|A61F|A61F|F16C|A61F,DE
2,1989,AT,US-4795646-A,A61K|A61K|A61P,AT
3,1989,AT,US-4795750-A,A61P|C07D|C07D|C07D,AT
4,1989,AT,US-4795967-A,G01F|G01F,AT
...,...,...,...,...,...
3617575,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201847-A1,A61G|A61G|A61B|A61G|A61G|A61G|A61G|A61G|A61G|A...,| | | | | | | | | | | | | | | ...
3617576,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201851-A1,A61G|A61G|A61G|A61G|G01G|A61B|A61G|A61G|A61G|A...,| | | | | | | | | | | | | | | ...
3617577,2008,US | US | US | US | US | US | US | US | US | U...,US-2008289108-A1,A61G|A61G|A61G|A61G|A61G|A61G|A61G|A61G|G01G|A...,| | | | | | | | | | | | | | | ...
3617578,2008,US | US | US | US | US | US | US | US | US | U...,US-7406731-B2,A61G|A61G|A61G|A61G|A61G|A61G|G01G|A61G|A61G|A...,US


**Corrected Final Codes for calculating frational counts and modes for inventor_country and assignee_country columns**



1. aggregate_unique(series)
Purpose: This function aggregates unique values in a Pandas series (a column of a DataFrame) into a single string, with each unique value separated by a pipe (|).
Operation:
series.dropna().unique(): Removes any NaN values from the series and then extracts the unique values.
'|'.join(...): Joins the unique values into a single string, with each value separated by a pipe (|).
2. calculate_fraction(countries_str)
Purpose: This function calculates the fractional share of each country code in a pipe-separated string and returns the result as a string, where each country code is followed by its fraction.
Operation:
countries_str.split('|'): Splits the string into a list of country codes using the pipe (|) as the delimiter.
countries.count(country) / total: Counts the occurrences of each unique country and divides it by the total number of countries to get the fractional share.
The result is returned as a pipe-separated string in the format country:share.
3. find_mode(countries_str)
Purpose: This function finds the mode (the most frequently occurring country code) from a pipe-separated string of country codes.
Operation:
pd.Series(...).mode(): Converts the list of country codes into a Pandas series and calculates the mode(s).
mode_series.tolist(): Converts the mode(s) into a list. If there are multiple modes, all are returned; if no mode exists, None is returned.
4. Aggregation and Application of Functions on the DataFrame
Purpose: The code then aggregates the DataFrame by publication_number and applies the functions defined above to calculate additional columns.
Operation:
df.groupby('publication_number').agg(...): Groups the DataFrame by publication_number and applies various aggregation functions.
aggregate_unique: Applied to inventor_country and assignee_country columns to create a pipe-separated string of unique country codes.
'first': Applied to publication_year and patents_cpcs to retain the first occurrence of each.
apply(calculate_fraction): Creates new columns inventors_country_frac and assignees_country_frac, which contain the fractional shares of each country code.
apply(find_mode): Creates new columns inventors_country_mode and assignees_country_mode, which contain the mode(s) of country codes.

In [ ]:
# Function to aggregate unique names and country codes into lists
def aggregate_unique(series):
    return '|'.join(series.dropna().unique())

# Function to calculate fractional shares from pipe-separated strings
def calculate_fraction(countries_str):
    if not countries_str:
        return {}
    countries = [country.strip() for country in countries_str.split('|')]  # Remove leading/trailing spaces
    total = len(countries)
    return '|'.join([f'{country}:{countries.count(country) / total:.4f}' for country in set(countries)])

# Function to find the mode of country codes
def find_mode(countries_str):
    if not countries_str:
        return None
    countries = pd.Series([country.strip() for country in countries_str.split('|')])
    mode_series = countries.mode()
    return mode_series.tolist() if not mode_series.empty else None

# Aggregate data by publication_number
aggregated_data = df.groupby('publication_number').agg({
    'inventor_country': aggregate_unique,
    'assignee_country': aggregate_unique,
    'publication_year': 'first',  # Retain the first occurrence
    'patents_cpcs': 'first'       # Retain the first occurrence
}).reset_index()

# Apply functions to calculate fractional shares and modes
aggregated_data['inventors_country_frac'] = aggregated_data['inventor_country'].apply(calculate_fraction)
aggregated_data['assignees_country_frac'] = aggregated_data['assignee_country'].apply(calculate_fraction)
aggregated_data['inventors_country_mode'] = aggregated_data['inventor_country'].apply(find_mode)
aggregated_data['assignees_country_mode'] = aggregated_data['assignee_country'].apply(find_mode)
aggregated_data

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode
0,US-10038-P,US,US,1997,A01D,US:1.0000,US:1.0000,[US],[US]
1,US-10298-P,AU,,1998,A01C,AU:1.0000,{},[AU],None
2,US-10791-P,DE,,1999,B61D,DE:1.0000,{},[DE],None
3,US-11104-P,JP | JP,JP,1999,F03G,JP:1.0000,JP:1.0000,[JP],[JP]
4,US-11240-P,US,,2000,D05B,US:1.0000,{},[US],None
...,...,...,...,...,...,...,...,...,...
3617575,US-RE40607-E,US | US,US,2008,A61B|A61B|A61B|A61B,US:1.0000,US:1.0000,[US],[US]
3617576,US-RE40608-E,GB | GB | GB,US,2008,G01S|A61B|A61B,GB:1.0000,US:1.0000,[GB],[US]
3617577,US-RE40609-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,[JP],[JP]
3617578,US-RE40610-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,[JP],[JP]


In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final.csv'
aggregated_data.to_csv(output_file_path, index=False)

In [ ]:
# Display the number of rows in the original and grouped DataFrames
print(f"Original DataFrame rows: {aggregated_data.shape[0]}")

Original DataFrame rows: 3617580


Load the created dataframe of aggregated_data into the name of df, as it is more efficient. The calculation of fractions and modes takes 54 minutes, so it is a good idea to just load it here for saving time.

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final.csv')
df

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode
0,US-10038-P,US,US,1997,A01D,US:1.0000,US:1.0000,['US'],['US']
1,US-10298-P,AU,NaN,1998,A01C,AU:1.0000,{},['AU'],NaN
2,US-10791-P,DE,NaN,1999,B61D,DE:1.0000,{},['DE'],NaN
3,US-11104-P,JP | JP,JP,1999,F03G,JP:1.0000,JP:1.0000,['JP'],['JP']
4,US-11240-P,US,NaN,2000,D05B,US:1.0000,{},['US'],NaN
...,...,...,...,...,...,...,...,...,...
3617575,US-RE40607-E,US | US,US,2008,A61B|A61B|A61B|A61B,US:1.0000,US:1.0000,['US'],['US']
3617576,US-RE40608-E,GB | GB | GB,US,2008,G01S|A61B|A61B,GB:1.0000,US:1.0000,['GB'],['US']
3617577,US-RE40609-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP']
3617578,US-RE40610-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP']


Create a 1000 random subset dataset based on random values of df for analysis: Optional

In [ ]:
# Create a subset of 1000 random rows
subset_df = df.sample(n=1000, random_state=42)

# Save the subset to a CSV file
subset_df.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/sample.csv', index=False)
subset_df

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode
1534130,US-4855944-A,US,US,1989,H03K,US:1.0000,US:1.0000,['US'],['US']
1074964,US-2007058378-A1,JP | JP,|,2007,H01R|H01R|F21V|F21S|F21S|F21V|G02B|H05K|F21V|G...,JP:1.0000,:1.0000,['JP'],['']
238019,US-2003044697-A1,JP,US,2003,G03F,JP:1.0000,US:1.0000,['JP'],['US']
2738261,US-6371268-B1,US | US | US | US | US | US | US | US | US | U...,US,2002,B60B|F16D,US:1.0000,US:1.0000,['US'],['US']
632487,US-2005049922-A1,US,US,2005,G07F|G07F|G07F|G07F,US:1.0000,US:1.0000,['US'],['US']
...,...,...,...,...,...,...,...,...,...
1093423,US-2007083666-A1,US,US,2007,H04N|H04N|H04N|H04N|H04N|H04N|H04N,US:1.0000,US:1.0000,['US'],['US']
1317075,US-2008089719-A1,DE | DE | DE,DE,2008,G03G|G03G|G03G|G03G|G03G|G03G|G03G,DE:1.0000,DE:1.0000,['DE'],['DE']
812827,US-2005289362-A1,US | US,|,2005,G06F|G06F,US:1.0000,:1.0000,['US'],['']
296854,US-2003119352-A1,US | US | US | US,NaN,2003,H01R|H01R,US:1.0000,{},['US'],NaN


**Generate a new column for fractional counts of cpc codes of patents**



1. calculate_fraction(codes_str)
Purpose: This function calculates the fractional share of each unique code in a pipe-separated string (codes_str) and returns a new pipe-separated string where each code is followed by its fractional share.

Steps:

Check if codes_str is empty or null:

if not codes_str: return {}: If the input string is empty or null, the function returns an empty dictionary.
Split and clean the string:

codes = [code.strip() for code in codes_str.split('|')]:
codes_str.split('|'): Splits the input string by the pipe (|) delimiter into a list of individual codes.
code.strip(): Removes any leading or trailing whitespace from each code in the list.
Calculate total number of codes:

total = len(codes): Counts the total number of codes in the list.
Calculate fractional shares:

return '|'.join([f'{code}:{codes.count(code) / total:.4f}' for code in set(codes)]):
set(codes): Creates a set of unique codes (removing duplicates).
codes.count(code) / total: Calculates the fraction of each unique code by counting its occurrences in the original list and dividing by the total number of codes.
f'{code}:{codes.count(code) / total:.4f}': Formats the code and its fractional share as a string with four decimal places.
'|'.join(...): Joins the formatted strings into a single pipe-separated string.
Example:

If codes_str = "A | B | A | C | A", the function would return "A:0.6000|B:0.2000|C:0.2000" because:
A occurs 3 times out of 5 (60%).
B occurs 1 time out of 5 (20%).
C occurs 1 time out of 5 (20%).
2. Applying the Function to the patents_cpcs Column
Operation:
df['patents_cpcs_frac'] = df['patents_cpcs'].apply(calculate_fraction):
This line applies the calculate_fraction function to each value in the patents_cpcs column of the DataFrame df.
The result is stored in a new column called patents_cpcs_frac.
Purpose:

The goal is to create a new column (patents_cpcs_frac) that represents the fractional distribution of CPC codes associated with each patent. This can provide insights into how the technological focus of each patent is distributed across different CPC codes.

In [ ]:
# Function to calculate fractional shares from pipe-separated strings
def calculate_fraction(codes_str):
    if not codes_str:
        return {}
    codes = [code.strip() for code in codes_str.split('|')]  # Remove leading/trailing spaces
    total = len(codes)
    return '|'.join([f'{code}:{codes.count(code) / total:.4f}' for code in set(codes)])

# Apply the function to calculate fractional shares for the patents_cpcs column
df['patents_cpcs_frac'] = df['patents_cpcs'].apply(calculate_fraction)
df

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_cpcs_frac
0,US-10038-P,US,US,1997,A01D,US:1.0000,US:1.0000,['US'],['US'],A01D:1.0000
1,US-10298-P,AU,NaN,1998,A01C,AU:1.0000,{},['AU'],NaN,A01C:1.0000
2,US-10791-P,DE,NaN,1999,B61D,DE:1.0000,{},['DE'],NaN,B61D:1.0000
3,US-11104-P,JP | JP,JP,1999,F03G,JP:1.0000,JP:1.0000,['JP'],['JP'],F03G:1.0000
4,US-11240-P,US,NaN,2000,D05B,US:1.0000,{},['US'],NaN,D05B:1.0000
...,...,...,...,...,...,...,...,...,...,...
3617575,US-RE40607-E,US | US,US,2008,A61B|A61B|A61B|A61B,US:1.0000,US:1.0000,['US'],['US'],A61B:1.0000
3617576,US-RE40608-E,GB | GB | GB,US,2008,G01S|A61B|A61B,GB:1.0000,US:1.0000,['GB'],['US'],A61B:0.6667|G01S:0.3333
3617577,US-RE40609-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333
3617578,US-RE40610-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333


In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final.csv'
df.to_csv(output_file_path, index=False)

In [ ]:
# Create a subset of 1000 random rows
subset_df = df.sample(n=1000, random_state=42)

# Save the subset to a CSV file
subset_df.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/sample.csv', index=False)
subset_df

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_cpcs_frac
1534130,US-4855944-A,US,US,1989,H03K,US:1.0000,US:1.0000,['US'],['US'],H03K:1.0000
1074964,US-2007058378-A1,JP | JP,|,2007,H01R|H01R|F21V|F21S|F21S|F21V|G02B|H05K|F21V|G...,JP:1.0000,:1.0000,['JP'],[''],H05K:0.0833|F21V:0.3333|G02B:0.1667|F21S:0.166...
238019,US-2003044697-A1,JP,US,2003,G03F,JP:1.0000,US:1.0000,['JP'],['US'],G03F:1.0000
2738261,US-6371268-B1,US | US | US | US | US | US | US | US | US | U...,US,2002,B60B|F16D,US:1.0000,US:1.0000,['US'],['US'],B60B:0.5000|F16D:0.5000
632487,US-2005049922-A1,US,US,2005,G07F|G07F|G07F|G07F,US:1.0000,US:1.0000,['US'],['US'],G07F:1.0000
...,...,...,...,...,...,...,...,...,...,...
1093423,US-2007083666-A1,US,US,2007,H04N|H04N|H04N|H04N|H04N|H04N|H04N,US:1.0000,US:1.0000,['US'],['US'],H04N:1.0000
1317075,US-2008089719-A1,DE | DE | DE,DE,2008,G03G|G03G|G03G|G03G|G03G|G03G|G03G,DE:1.0000,DE:1.0000,['DE'],['DE'],G03G:1.0000
812827,US-2005289362-A1,US | US,|,2005,G06F|G06F,US:1.0000,:1.0000,['US'],[''],G06F:1.0000
296854,US-2003119352-A1,US | US | US | US,NaN,2003,H01R|H01R,US:1.0000,{},['US'],NaN,H01R:1.0000


In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final.csv')
df

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_cpcs_frac
0,US-10038-P,US,US,1997,A01D,US:1.0000,US:1.0000,['US'],['US'],A01D:1.0000
1,US-10298-P,AU,NaN,1998,A01C,AU:1.0000,{},['AU'],NaN,A01C:1.0000
2,US-10791-P,DE,NaN,1999,B61D,DE:1.0000,{},['DE'],NaN,B61D:1.0000
3,US-11104-P,JP | JP,JP,1999,F03G,JP:1.0000,JP:1.0000,['JP'],['JP'],F03G:1.0000
4,US-11240-P,US,NaN,2000,D05B,US:1.0000,{},['US'],NaN,D05B:1.0000
...,...,...,...,...,...,...,...,...,...,...
3617575,US-RE40607-E,US | US,US,2008,A61B|A61B|A61B|A61B,US:1.0000,US:1.0000,['US'],['US'],A61B:1.0000
3617576,US-RE40608-E,GB | GB | GB,US,2008,G01S|A61B|A61B,GB:1.0000,US:1.0000,['GB'],['US'],A61B:0.6667|G01S:0.3333
3617577,US-RE40609-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333
3617578,US-RE40610-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333


In [ ]:
# Load the mapping dataset
mapping_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/Patents_CPCs_Final.csv')
mapping_df


,Unnamed: 0,patents_ipcs,patents_cpcs,Consistency,ID,sector,sector_tech,tech_sech
0,1,B64B,B64B,True,1,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
1,2,B64C,B64C,True,2,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
2,3,B64D,B64D,True,3,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
3,4,B64F,B64F,True,4,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
4,5,B64G,B64G,True,5,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
...,...,...,...,...,...,...,...,...
444,445,B61L,B61L,True,445,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
445,446,B62M,B62M,True,446,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
446,447,B61G,B61G,True,447,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
447,448,B62B,B62B,True,448,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech


1. Creating a Dictionary to Map CPC Codes to Sectors
The first part of the code creates a dictionary that links specific CPC (Cooperative Patent Classification) codes to their corresponding technology sectors. This dictionary is built using two lists: one containing CPC codes and the other containing sector names. By pairing each CPC code with its sector, the dictionary allows for easy lookup of the sector based on the CPC code.

Example: If you have the following lists:

CPC codes: ["A01B", "C07D", "H04L"]
Sectors: ["Agriculture", "Chemistry", "Telecommunications"]
The dictionary will look like this:

"A01B" maps to "Agriculture"
"C07D" maps to "Chemistry"
"H04L" maps to "Telecommunications"
2. Mapping CPC Codes in a String to Sectors
The next part of the code defines a function that takes a string of CPC codes (separated by a pipe |), looks up each code in the dictionary, and returns a string of the corresponding sectors (also separated by a pipe).

Example: If the input string is "A01B|C07D|H04L", the function will:

Look up "A01B" and find it maps to "Agriculture"
Look up "C07D" and find it maps to "Chemistry"
Look up "H04L" and find it maps to "Telecommunications"
Return the string "Agriculture|Chemistry|Telecommunications"
3. Applying the Sector Mapping Function
The function from step 2 is then applied to a column in a DataFrame (which contains strings of CPC codes). This creates a new column in the DataFrame where each string of CPC codes is replaced with a string of corresponding sectors.

Example: If a row in the DataFrame has the CPC string "A01B|C07D", after applying the function, the new column will contain "Agriculture|Chemistry".

4. Calculating Sector Fractions
Next, a more complex function is defined to calculate the fractional share of each sector. This function considers both the share of each CPC code (how much of the patent's focus is in that area) and the share of inventors from different countries. The function calculates how much each country contributes to each sector, based on the CPC code shares.

Example: If a row indicates that:

50% of the patent's focus is on "A01B" (Agriculture) and 50% on "C07D" (Chemistry).
70% of the inventors are from the US and 30% from Germany.
The function calculates:

For Agriculture: US inventors contribute 35% (50% of 70%) and German inventors contribute 15% (50% of 30%).
For Chemistry: US inventors contribute 35% and German inventors contribute 15%.
So, the function would return:

"Agriculture:0.5000|Chemistry:0.5000"
5. Applying the Sector Fraction Calculation
Finally, the function from step 4 is applied to each row in the DataFrame to compute the sector fractions based on the CPC and inventor country shares. This creates another new column in the DataFrame, showing the calculated sector fractions for each patent.

Example: If a row has:

50% focus on "A01B" (Agriculture) and 50% on "C07D" (Chemistry).
70% US inventors and 30% German inventors.
The resulting sector fraction column will show:

"Agriculture:0.5000|Chemistry:0.5000"
This process allows for a detailed analysis of how different sectors are represented in the patents, taking into account both the technological focus and the geographical distribution of inventors.

In [ ]:
# Create a dictionary to map CPC codes to sectors
cpc_to_sector = dict(zip(mapping_df['patents_cpcs'], mapping_df['sector_tech']))

# Function to map CPC codes to sectors
def map_cpcs_to_sectors(cpc_str):
    cpcs = cpc_str.split('|')
    sectors = [cpc_to_sector.get(cpc.strip(), 'Unknown') for cpc in cpcs]
    return '|'.join(sectors)

# Apply the function to the patents_cpcs column
df['sectors'] = df['patents_cpcs'].apply(map_cpcs_to_sectors)

# Function to calculate sector fractions based on CPC fractions and inventor fractions
def calculate_sector_fraction(row):
    cpc_fractions = row['patents_cpcs_frac'].split('|')
    inventor_fractions = row['inventors_country_frac'].split('|')

    sector_fractions = {}
    for cpc_frac in cpc_fractions:
        cpc_code, cpc_share = cpc_frac.split(':')
        cpc_share = float(cpc_share)

        sector = cpc_to_sector.get(cpc_code.strip(), 'Unknown')

        if sector not in sector_fractions:
            sector_fractions[sector] = 0

        for inv_frac in inventor_fractions:
            inv_country, inv_share = inv_frac.split(':')
            inv_share = float(inv_share)
            sector_fractions[sector] += cpc_share * inv_share

    return '|'.join([f'{sector}:{share:.4f}' for sector, share in sector_fractions.items()])

# Apply the function to calculate sector fractions
df['sector_fractions'] = df.apply(calculate_sector_fraction, axis=1)
df


,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_cpcs_frac,sectors,sector_fractions
0,US-10038-P,US,US,1997,A01D,US:1.0000,US:1.0000,['US'],['US'],A01D:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
1,US-10298-P,AU,NaN,1998,A01C,AU:1.0000,{},['AU'],NaN,A01C:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
2,US-10791-P,DE,NaN,1999,B61D,DE:1.0000,{},['DE'],NaN,B61D:1.0000,Rail_and_transport_equipment,Rail_and_transport_equipment:1.0000
3,US-11104-P,JP | JP,JP,1999,F03G,JP:1.0000,JP:1.0000,['JP'],['JP'],F03G:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
4,US-11240-P,US,NaN,2000,D05B,US:1.0000,{},['US'],NaN,D05B:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
3617575,US-RE40607-E,US | US,US,2008,A61B|A61B|A61B|A61B,US:1.0000,US:1.0000,['US'],['US'],A61B:1.0000,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:1.0000
3617576,US-RE40608-E,GB | GB | GB,US,2008,G01S|A61B|A61B,GB:1.0000,US:1.0000,['GB'],['US'],A61B:0.6667|G01S:0.3333,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:1.0000
3617577,US-RE40609-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:0.6667|M...
3617578,US-RE40610-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:0.6667|M...


In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final_final.csv'
df.to_csv(output_file_path, index=False)

In [15]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final_final.csv')
df

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_cpcs_frac,sectors,sector_fractions
0,US-10038-P,US,US,1997,A01D,US:1.0000,US:1.0000,['US'],['US'],A01D:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
1,US-10298-P,AU,NaN,1998,A01C,AU:1.0000,{},['AU'],NaN,A01C:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
2,US-10791-P,DE,NaN,1999,B61D,DE:1.0000,{},['DE'],NaN,B61D:1.0000,Rail_and_transport_equipment,Rail_and_transport_equipment:1.0000
3,US-11104-P,JP | JP,JP,1999,F03G,JP:1.0000,JP:1.0000,['JP'],['JP'],F03G:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
4,US-11240-P,US,NaN,2000,D05B,US:1.0000,{},['US'],NaN,D05B:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
3617575,US-RE40607-E,US | US,US,2008,A61B|A61B|A61B|A61B,US:1.0000,US:1.0000,['US'],['US'],A61B:1.0000,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:1.0000
3617576,US-RE40608-E,GB | GB | GB,US,2008,G01S|A61B|A61B,GB:1.0000,US:1.0000,['GB'],['US'],A61B:0.6667|G01S:0.3333,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:1.0000
3617577,US-RE40609-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:0.6667|M...
3617578,US-RE40610-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:0.6667|M...


In [16]:
# Create a subset of 1000 random rows
subset_df = df.sample(n=1000, random_state=42)

# Save the subset to a CSV file
subset_df.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/sample_final_final_final_final.csv', index=False)
subset_df

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_cpcs_frac,sectors,sector_fractions
1534130,US-4855944-A,US,US,1989,H03K,US:1.0000,US:1.0000,['US'],['US'],H03K:1.0000,Office_accounting_and_computing_machinery,Office_accounting_and_computing_machinery:1.0000
1074964,US-2007058378-A1,JP | JP,|,2007,H01R|H01R|F21V|F21S|F21S|F21V|G02B|H05K|F21V|G...,JP:1.0000,:1.0000,['JP'],[''],H05K:0.0833|F21V:0.3333|G02B:0.1667|F21S:0.166...,Electrical_machinery_and_apparatus|Electrical_...,Radio_TV_and_communications_equipment:0.1666|E...
238019,US-2003044697-A1,JP,US,2003,G03F,JP:1.0000,US:1.0000,['JP'],['US'],G03F:1.0000,Medical_precision_and_optical_instruments,Medical_precision_and_optical_instruments:1.0000
2738261,US-6371268-B1,US | US | US | US | US | US | US | US | US | U...,US,2002,B60B|F16D,US:1.0000,US:1.0000,['US'],['US'],B60B:0.5000|F16D:0.5000,Motor_Vehicles_trailers_and_semi_trailers|Mach...,Motor_Vehicles_trailers_and_semi_trailers:0.50...
632487,US-2005049922-A1,US,US,2005,G07F|G07F|G07F|G07F,US:1.0000,US:1.0000,['US'],['US'],G07F:1.0000,Office_accounting_and_computing_machinery|Offi...,Office_accounting_and_computing_machinery:1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
1093423,US-2007083666-A1,US,US,2007,H04N|H04N|H04N|H04N|H04N|H04N|H04N,US:1.0000,US:1.0000,['US'],['US'],H04N:1.0000,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:1.0000
1317075,US-2008089719-A1,DE | DE | DE,DE,2008,G03G|G03G|G03G|G03G|G03G|G03G|G03G,DE:1.0000,DE:1.0000,['DE'],['DE'],G03G:1.0000,Office_accounting_and_computing_machinery|Offi...,Office_accounting_and_computing_machinery:1.0000
812827,US-2005289362-A1,US | US,|,2005,G06F|G06F,US:1.0000,:1.0000,['US'],[''],G06F:1.0000,Office_accounting_and_computing_machinery|Offi...,Office_accounting_and_computing_machinery:1.0000
296854,US-2003119352-A1,US | US | US | US,NaN,2003,H01R|H01R,US:1.0000,{},['US'],NaN,H01R:1.0000,Electrical_machinery_and_apparatus|Electrical_...,Electrical_machinery_and_apparatus:1.0000


In [17]:
def calculate_inventor_sector_fraction(row):
    inventor_fractions = row['inventors_country_frac'].split('|')
    cpc_fractions = row['patents_cpcs_frac'].split('|')

    sector_fractions = []
    for inv_frac in inventor_fractions:
        inv_country, inv_share = inv_frac.split(':')
        inv_share = float(inv_share)

        for cpc_frac in cpc_fractions:
            cpc_code, cpc_share = cpc_frac.split(':')
            cpc_share = float(cpc_share)

            # Multiply the shares
            combined_share = inv_share * cpc_share
            sector_fractions.append(f'{inv_country.strip()} {cpc_code.strip()} {combined_share:.6f}')

    return ' | '.join(sector_fractions)

# Apply the function to create the new column
df['inventor_sector_fraction'] = df.apply(calculate_inventor_sector_fraction, axis=1)

# Display the DataFrame with the new column
df


,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_cpcs_frac,sectors,sector_fractions,inventor_sector_fraction
0,US-10038-P,US,US,1997,A01D,US:1.0000,US:1.0000,['US'],['US'],A01D:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000,US A01D 1.000000
1,US-10298-P,AU,NaN,1998,A01C,AU:1.0000,{},['AU'],NaN,A01C:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000,AU A01C 1.000000
2,US-10791-P,DE,NaN,1999,B61D,DE:1.0000,{},['DE'],NaN,B61D:1.0000,Rail_and_transport_equipment,Rail_and_transport_equipment:1.0000,DE B61D 1.000000
3,US-11104-P,JP | JP,JP,1999,F03G,JP:1.0000,JP:1.0000,['JP'],['JP'],F03G:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000,JP F03G 1.000000
4,US-11240-P,US,NaN,2000,D05B,US:1.0000,{},['US'],NaN,D05B:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000,US D05B 1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3617575,US-RE40607-E,US | US,US,2008,A61B|A61B|A61B|A61B,US:1.0000,US:1.0000,['US'],['US'],A61B:1.0000,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:1.0000,US A61B 1.000000
3617576,US-RE40608-E,GB | GB | GB,US,2008,G01S|A61B|A61B,GB:1.0000,US:1.0000,['GB'],['US'],A61B:0.6667|G01S:0.3333,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:1.0000,GB A61B 0.666700 | GB G01S 0.333300
3617577,US-RE40609-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:0.6667|M...,JP H04N 0.666700 | JP G02B 0.333300
3617578,US-RE40610-E,JP,JP,2008,H04N|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],H04N:0.6667|G02B:0.3333,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:0.6667|M...,JP H04N 0.666700 | JP G02B 0.333300


In [18]:
# Create a subset of 1000 random rows
subset_df = df.sample(n=1000, random_state=42)

# Save the subset to a CSV file
subset_df.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/sample_final_final_final_final_final.csv', index=False)
subset_df

,publication_number,inventor_country,assignee_country,publication_year,patents_cpcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_cpcs_frac,sectors,sector_fractions,inventor_sector_fraction
1534130,US-4855944-A,US,US,1989,H03K,US:1.0000,US:1.0000,['US'],['US'],H03K:1.0000,Office_accounting_and_computing_machinery,Office_accounting_and_computing_machinery:1.0000,US H03K 1.000000
1074964,US-2007058378-A1,JP | JP,|,2007,H01R|H01R|F21V|F21S|F21S|F21V|G02B|H05K|F21V|G...,JP:1.0000,:1.0000,['JP'],[''],H05K:0.0833|F21V:0.3333|G02B:0.1667|F21S:0.166...,Electrical_machinery_and_apparatus|Electrical_...,Radio_TV_and_communications_equipment:0.1666|E...,JP H05K 0.083300 | JP F21V 0.333300 | JP G02B ...
238019,US-2003044697-A1,JP,US,2003,G03F,JP:1.0000,US:1.0000,['JP'],['US'],G03F:1.0000,Medical_precision_and_optical_instruments,Medical_precision_and_optical_instruments:1.0000,JP G03F 1.000000
2738261,US-6371268-B1,US | US | US | US | US | US | US | US | US | U...,US,2002,B60B|F16D,US:1.0000,US:1.0000,['US'],['US'],B60B:0.5000|F16D:0.5000,Motor_Vehicles_trailers_and_semi_trailers|Mach...,Motor_Vehicles_trailers_and_semi_trailers:0.50...,US B60B 0.500000 | US F16D 0.500000
632487,US-2005049922-A1,US,US,2005,G07F|G07F|G07F|G07F,US:1.0000,US:1.0000,['US'],['US'],G07F:1.0000,Office_accounting_and_computing_machinery|Offi...,Office_accounting_and_computing_machinery:1.0000,US G07F 1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093423,US-2007083666-A1,US,US,2007,H04N|H04N|H04N|H04N|H04N|H04N|H04N,US:1.0000,US:1.0000,['US'],['US'],H04N:1.0000,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:1.0000,US H04N 1.000000
1317075,US-2008089719-A1,DE | DE | DE,DE,2008,G03G|G03G|G03G|G03G|G03G|G03G|G03G,DE:1.0000,DE:1.0000,['DE'],['DE'],G03G:1.0000,Office_accounting_and_computing_machinery|Offi...,Office_accounting_and_computing_machinery:1.0000,DE G03G 1.000000
812827,US-2005289362-A1,US | US,|,2005,G06F|G06F,US:1.0000,:1.0000,['US'],[''],G06F:1.0000,Office_accounting_and_computing_machinery|Offi...,Office_accounting_and_computing_machinery:1.0000,US G06F 1.000000
296854,US-2003119352-A1,US | US | US | US,NaN,2003,H01R|H01R,US:1.0000,{},['US'],NaN,H01R:1.0000,Electrical_machinery_and_apparatus|Electrical_...,Electrical_machinery_and_apparatus:1.0000,US H01R 1.000000


In [19]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final_final_final.csv'
df.to_csv(output_file_path, index=False)

In [20]:
# Function to extract and split the inventor_sector_fraction into separate rows
def extract_fractions(row):
    fractions = row['inventor_sector_fraction'].split(' | ')
    data = []
    for fraction in fractions:
        parts = fraction.split(' ')
        country = parts[0]
        cpc_code = parts[1]
        frac_value = float(parts[2])
        year = row['publication_year']
        data.append([country, year, cpc_code, frac_value])
    return data

# Initialize an empty list to hold the new dataset rows
new_data = []

# Apply the function to each row and extend the new_data list
for index, row in df.iterrows():
    new_data.extend(extract_fractions(row))

# Convert the list to a DataFrame
new_dataset = pd.DataFrame(new_data, columns=['country', 'year', 'CPC_code', 'fraction'])

# Group by country, year, and CPC_code, and sum the fractions
aggregated_dataset = new_dataset.groupby(['country', 'year', 'CPC_code']).agg({
    'fraction': 'sum'
}).reset_index()

# Rename the fraction column to 'country_year_cpc_fraction'
aggregated_dataset.rename(columns={'fraction': 'country_year_cpc_fraction'}, inplace=True)

# Display the first few rows of the aggregated dataset
aggregated_dataset


,country,year,CPC_code,country_year_cpc_fraction
0,AT,1989,A01G,2.000000
1,AT,1989,A01N,1.000000
2,AT,1989,A47J,0.361089
3,AT,1989,A61B,3.277025
4,AT,1989,A61F,5.533350
...,...,...,...,...
79104,US,2008,H05C,6.842900
79105,US,2008,H05F,8.352425
79106,US,2008,H05G,48.902250
79107,US,2008,H05H,115.479135


In [21]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final_final_final_final.csv'
aggregated_dataset.to_csv(output_file_path, index=False)

In [22]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final_final_final_final.csv')
df

,country,year,CPC_code,country_year_cpc_fraction
0,AT,1989,A01G,2.000000
1,AT,1989,A01N,1.000000
2,AT,1989,A47J,0.361089
3,AT,1989,A61B,3.277025
4,AT,1989,A61F,5.533350
...,...,...,...,...
79104,US,2008,H05C,6.842900
79105,US,2008,H05F,8.352425
79106,US,2008,H05G,48.902250
79107,US,2008,H05H,115.479135


In [23]:
import pandas as pd
import itertools

# Assuming `df` is your current DataFrame

# 1. Define the unique values for country, year, and CPC_code
unique_countries = df['country'].unique()
unique_years = df['year'].unique()
unique_cpc_codes = df['CPC_code'].unique()

# 2. Generate all possible combinations of country, year, and CPC_code
all_combinations = pd.DataFrame(list(itertools.product(unique_countries, unique_years, unique_cpc_codes)),
                                columns=['country', 'year', 'CPC_code'])

# 3. Merge the all_combinations DataFrame with the existing df
merged_df = pd.merge(all_combinations, df,
                     on=['country', 'year', 'CPC_code'],
                     how='left')

# 4. Replace NaN values in the 'country_year_cpc_fraction' column with 0
merged_df['country_year_cpc_fraction'].fillna(0, inplace=True)

# 5. Optional: Reorder the columns if needed
merged_df = merged_df[['country', 'year', 'CPC_code', 'country_year_cpc_fraction']]

# 6. Save the result to a new CSV file
merged_df.to_csv('complete_aggregated_data.csv', index=False)

# Display the first few rows to check the result
merged_df



,country,year,CPC_code,country_year_cpc_fraction
0,AT,1989,A01G,2.000000
1,AT,1989,A01N,1.000000
2,AT,1989,A47J,0.361089
3,AT,1989,A61B,3.277025
4,AT,1989,A61F,5.533350
...,...,...,...,...
129295,US,2008,B61J,2.500000
129296,US,2008,B41G,0.750000
129297,US,2008,B41D,0.000000
129298,US,2008,G21J,0.000000


In [24]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final_final_final_final_final.csv'
merged_df.to_csv(output_file_path, index=False)

In [25]:
import pandas as pd

# Load the mapping dataset
mapping_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/Patents_CPCs_Final.csv')

# Perform the merge to add sector_tech and tech_sech to df
df = pd.merge(df, mapping_df[['patents_cpcs', 'sector_tech', 'tech_sech']],
              left_on='CPC_code', right_on='patents_cpcs',
              how='left')

# Drop the extra 'patents_cpcs' column after the merge if it's not needed
df.drop(columns=['patents_cpcs'], inplace=True)

df


,country,year,CPC_code,country_year_cpc_fraction,sector_tech,tech_sech
0,AT,1989,A01G,2.000000,Machinery_and_equipment,medium_tech
1,AT,1989,A01N,1.000000,Chemicals_excluding_pharmaceuticals,medium_tech
2,AT,1989,A47J,0.361089,Machinery_and_equipment,medium_tech
3,AT,1989,A61B,3.277025,Medical_precision_and_optical_instruments,high_tech
4,AT,1989,A61F,5.533350,Medical_precision_and_optical_instruments,high_tech
...,...,...,...,...,...,...
79694,US,2008,H05C,6.842900,Electrical_machinery_and_apparatus,medium_tech
79695,US,2008,H05F,8.352425,Machinery_and_equipment,medium_tech
79696,US,2008,H05G,48.902250,Medical_precision_and_optical_instruments,high_tech
79697,US,2008,H05H,115.479135,Machinery_and_equipment,medium_tech


In [26]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/aggregated_data_final_final_final_final_final_final_final_final.csv'
df.to_csv(output_file_path, index=False)

In [27]:
import pandas as pd


# 1. High Tech (1991-1998)
high_tech_1991_1998 = df[(df['year'].between(1991, 1998)) & (df['tech_sech'] == 'high_tech')]
high_tech_1991_1998.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/high_tech_1991_1998.csv', index=False)

# 2. Medium Tech (1991-1998)
medium_tech_1991_1998 = df[(df['year'].between(1991, 1998)) & (df['tech_sech'] == 'medium_tech')]
medium_tech_1991_1998.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/medium_tech_1991_1998.csv', index=False)

# 3. High Tech (2000-2007)
high_tech_2000_2007 = df[(df['year'].between(2000, 2007)) & (df['tech_sech'] == 'high_tech')]
high_tech_2000_2007.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/high_tech_2000_2007.csv', index=False)

# 4. Medium Tech (2000-2007)
medium_tech_2000_2007 = df[(df['year'].between(2000, 2007)) & (df['tech_sech'] == 'medium_tech')]
medium_tech_2000_2007.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/medium_tech_2000_2007.csv', index=False)



In [28]:
import pandas as pd

# Assuming high_tech_1991_1998 is already defined and contains the relevant data

# 1. Sum by country, sector_tech, and year to get country_sector_patents
country_sector_patents = high_tech_1991_1998.groupby(['country', 'sector_tech', 'year'])['country_year_cpc_fraction'].sum().reset_index()
country_sector_patents.rename(columns={'country_year_cpc_fraction': 'country_sector_patents'}, inplace=True)

# 2. Sum by country and year to get country_total_patents
country_total_patents = high_tech_1991_1998.groupby(['country', 'year'])['country_year_cpc_fraction'].sum().reset_index()
country_total_patents.rename(columns={'country_year_cpc_fraction': 'country_total_patents'}, inplace=True)

# 3. Sum by sector_tech and year to get world_sector_patents
world_sector_patents = high_tech_1991_1998.groupby(['sector_tech', 'year'])['country_year_cpc_fraction'].sum().reset_index()
world_sector_patents.rename(columns={'country_year_cpc_fraction': 'world_sector_patents'}, inplace=True)

# 4. Sum by year to get world_total_patents
world_total_patents = high_tech_1991_1998.groupby(['year'])['country_year_cpc_fraction'].sum().reset_index()
world_total_patents.rename(columns={'country_year_cpc_fraction': 'world_total_patents'}, inplace=True)

# Merge these aggregated columns back into the original high_tech_1991_1998 dataframe
high_tech_1991_1998 = pd.merge(high_tech_1991_1998, country_sector_patents, on=['country', 'sector_tech', 'year'], how='left')
high_tech_1991_1998 = pd.merge(high_tech_1991_1998, country_total_patents, on=['country', 'year'], how='left')
high_tech_1991_1998 = pd.merge(high_tech_1991_1998, world_sector_patents, on=['sector_tech', 'year'], how='left')
high_tech_1991_1998 = pd.merge(high_tech_1991_1998, world_total_patents, on=['year'], how='left')

# Calculate RTA: (country_sector_patents / country_total_patents) / (world_sector_patents / world_total_patents)
high_tech_1991_1998['RTA'] = (high_tech_1991_1998['country_sector_patents'] / high_tech_1991_1998['country_total_patents']) / (high_tech_1991_1998['world_sector_patents'] / high_tech_1991_1998['world_total_patents'])

# Calculate Normalized RTA: (RTA - 1) / (RTA + 1)
high_tech_1991_1998['Normalized_RTA'] = (high_tech_1991_1998['RTA'] - 1) / (high_tech_1991_1998['RTA'] + 1)

# Optionally, save the RTA-calculated dataset to a CSV file
high_tech_1991_1998.to_csv('high_tech_1991_1998_rta.csv', index=False)

# Display the first few rows to verify the RTA and Normalized RTA calculation
high_tech_1991_1998



,country,year,CPC_code,country_year_cpc_fraction,sector_tech,tech_sech,country_sector_patents,country_total_patents,world_sector_patents,world_total_patents,RTA,Normalized_RTA
0,AT,1991,A61B,1.133300,Medical_precision_and_optical_instruments,high_tech,39.522311,91.798811,13320.489225,36726.006939,1.187022,0.085515
1,AT,1991,A61C,2.666700,Medical_precision_and_optical_instruments,high_tech,39.522311,91.798811,13320.489225,36726.006939,1.187022,0.085515
2,AT,1991,A61F,2.200000,Medical_precision_and_optical_instruments,high_tech,39.522311,91.798811,13320.489225,36726.006939,1.187022,0.085515
3,AT,1991,A61G,1.000000,Medical_precision_and_optical_instruments,high_tech,39.522311,91.798811,13320.489225,36726.006939,1.187022,0.085515
4,AT,1991,A61K,7.170950,Pharmaceuticals,high_tech,18.622100,91.798811,5035.739516,36726.006939,1.479456,0.193371
...,...,...,...,...,...,...,...,...,...,...,...,...
8839,US,1998,H04Q,317.345146,Radio_TV_and_communications_equipment,high_tech,11531.654806,43071.504918,21300.685625,71902.062696,0.903752,-0.050557
8840,US,1998,H04R,198.522687,Radio_TV_and_communications_equipment,high_tech,11531.654806,43071.504918,21300.685625,71902.062696,0.903752,-0.050557
8841,US,1998,H04S,33.650400,Radio_TV_and_communications_equipment,high_tech,11531.654806,43071.504918,21300.685625,71902.062696,0.903752,-0.050557
8842,US,1998,H05G,15.209000,Medical_precision_and_optical_instruments,high_tech,13376.622883,43071.504918,20584.225952,71902.062696,1.084834,0.040691


In [29]:
high_tech_1991_1998.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/high_tech_1991_1998_RTA.csv', index=False)


In [30]:
import pandas as pd

# Assuming medium_tech_1991_1998 is already defined and contains the relevant data

# 1. Sum by country, sector_tech, and year to get country_sector_patents
country_sector_patents = medium_tech_1991_1998.groupby(['country', 'sector_tech', 'year'])['country_year_cpc_fraction'].sum().reset_index()
country_sector_patents.rename(columns={'country_year_cpc_fraction': 'country_sector_patents'}, inplace=True)

# 2. Sum by country and year to get country_total_patents
country_total_patents = medium_tech_1991_1998.groupby(['country', 'year'])['country_year_cpc_fraction'].sum().reset_index()
country_total_patents.rename(columns={'country_year_cpc_fraction': 'country_total_patents'}, inplace=True)

# 3. Sum by sector_tech and year to get world_sector_patents
world_sector_patents = medium_tech_1991_1998.groupby(['sector_tech', 'year'])['country_year_cpc_fraction'].sum().reset_index()
world_sector_patents.rename(columns={'country_year_cpc_fraction': 'world_sector_patents'}, inplace=True)

# 4. Sum by year to get world_total_patents
world_total_patents = medium_tech_1991_1998.groupby(['year'])['country_year_cpc_fraction'].sum().reset_index()
world_total_patents.rename(columns={'country_year_cpc_fraction': 'world_total_patents'}, inplace=True)

# Merge these aggregated columns back into the original medium_tech_1991_1998 dataframe
medium_tech_1991_1998 = pd.merge(medium_tech_1991_1998, country_sector_patents, on=['country', 'sector_tech', 'year'], how='left')
medium_tech_1991_1998 = pd.merge(medium_tech_1991_1998, country_total_patents, on=['country', 'year'], how='left')
medium_tech_1991_1998 = pd.merge(medium_tech_1991_1998, world_sector_patents, on=['sector_tech', 'year'], how='left')
medium_tech_1991_1998 = pd.merge(medium_tech_1991_1998, world_total_patents, on=['year'], how='left')

# Calculate RTA: (country_sector_patents / country_total_patents) / (world_sector_patents / world_total_patents)
medium_tech_1991_1998['RTA'] = (medium_tech_1991_1998['country_sector_patents'] / medium_tech_1991_1998['country_total_patents']) / (medium_tech_1991_1998['world_sector_patents'] / medium_tech_1991_1998['world_total_patents'])

# Calculate Normalized RTA: (RTA - 1) / (RTA + 1)
medium_tech_1991_1998['Normalized_RTA'] = (medium_tech_1991_1998['RTA'] - 1) / (medium_tech_1991_1998['RTA'] + 1)

# Optionally, save the RTA-calculated dataset to a CSV file
medium_tech_1991_1998.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/medium_tech_1991_1998_RTA.csv', index=False)

# Display the first few rows to verify the RTA and Normalized RTA calculation
medium_tech_1991_1998


,country,year,CPC_code,country_year_cpc_fraction,sector_tech,tech_sech,country_sector_patents,country_total_patents,world_sector_patents,world_total_patents,RTA,Normalized_RTA
0,AT,1991,A01D,1.00000,Machinery_and_equipment,medium_tech,83.420640,170.832505,19852.014854,40899.923854,1.006053,0.003017
1,AT,1991,A01G,0.25000,Machinery_and_equipment,medium_tech,83.420640,170.832505,19852.014854,40899.923854,1.006053,0.003017
2,AT,1991,A01K,1.00000,Machinery_and_equipment,medium_tech,83.420640,170.832505,19852.014854,40899.923854,1.006053,0.003017
3,AT,1991,A01M,0.40000,Machinery_and_equipment,medium_tech,83.420640,170.832505,19852.014854,40899.923854,1.006053,0.003017
4,AT,1991,A01N,0.70000,Chemicals_excluding_pharmaceuticals,medium_tech,39.316700,170.832505,10669.982600,40899.923854,0.882197,-0.062588
...,...,...,...,...,...,...,...,...,...,...,...,...
19522,US,1998,H02P,114.81686,Electrical_machinery_and_apparatus,medium_tech,3138.248371,27189.041230,5502.877597,47600.276747,0.998419,-0.000791
19523,US,1998,H05B,278.80385,Machinery_and_equipment,medium_tech,13608.270349,27189.041230,22507.855657,47600.276747,1.058484,0.028411
19524,US,1998,H05C,1.50000,Electrical_machinery_and_apparatus,medium_tech,3138.248371,27189.041230,5502.877597,47600.276747,0.998419,-0.000791
19525,US,1998,H05F,8.22770,Machinery_and_equipment,medium_tech,13608.270349,27189.041230,22507.855657,47600.276747,1.058484,0.028411


In [31]:
import pandas as pd

# Assuming high_tech_2000_2007 is already defined and contains the relevant data

# 1. Sum by country, sector_tech, and year to get country_sector_patents
country_sector_patents = high_tech_2000_2007.groupby(['country', 'sector_tech', 'year'])['country_year_cpc_fraction'].sum().reset_index()
country_sector_patents.rename(columns={'country_year_cpc_fraction': 'country_sector_patents'}, inplace=True)

# 2. Sum by country and year to get country_total_patents
country_total_patents = high_tech_2000_2007.groupby(['country', 'year'])['country_year_cpc_fraction'].sum().reset_index()
country_total_patents.rename(columns={'country_year_cpc_fraction': 'country_total_patents'}, inplace=True)

# 3. Sum by sector_tech and year to get world_sector_patents
world_sector_patents = high_tech_2000_2007.groupby(['sector_tech', 'year'])['country_year_cpc_fraction'].sum().reset_index()
world_sector_patents.rename(columns={'country_year_cpc_fraction': 'world_sector_patents'}, inplace=True)

# 4. Sum by year to get world_total_patents
world_total_patents = high_tech_2000_2007.groupby(['year'])['country_year_cpc_fraction'].sum().reset_index()
world_total_patents.rename(columns={'country_year_cpc_fraction': 'world_total_patents'}, inplace=True)

# Merge these aggregated columns back into the original high_tech_2000_2007 dataframe
high_tech_2000_2007 = pd.merge(high_tech_2000_2007, country_sector_patents, on=['country', 'sector_tech', 'year'], how='left')
high_tech_2000_2007 = pd.merge(high_tech_2000_2007, country_total_patents, on=['country', 'year'], how='left')
high_tech_2000_2007 = pd.merge(high_tech_2000_2007, world_sector_patents, on=['sector_tech', 'year'], how='left')
high_tech_2000_2007 = pd.merge(high_tech_2000_2007, world_total_patents, on=['year'], how='left')

# Calculate RTA: (country_sector_patents / country_total_patents) / (world_sector_patents / world_total_patents)
high_tech_2000_2007['RTA'] = (high_tech_2000_2007['country_sector_patents'] / high_tech_2000_2007['country_total_patents']) / (high_tech_2000_2007['world_sector_patents'] / high_tech_2000_2007['world_total_patents'])

# Calculate Normalized RTA: (RTA - 1) / (RTA + 1)
high_tech_2000_2007['Normalized_RTA'] = (high_tech_2000_2007['RTA'] - 1) / (high_tech_2000_2007['RTA'] + 1)

# Optionally, save the RTA-calculated dataset to a CSV file
high_tech_2000_2007.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/high_tech_2000_2007_RTA.csv', index=False)

# Display the first few rows to verify the RTA and Normalized RTA calculation
high_tech_2000_2007


,country,year,CPC_code,country_year_cpc_fraction,sector_tech,tech_sech,country_sector_patents,country_total_patents,world_sector_patents,world_total_patents,RTA,Normalized_RTA
0,AT,2000,A61B,5.309720,Medical_precision_and_optical_instruments,high_tech,59.625778,183.423946,20260.111317,75042.868275,1.204053,0.092581
1,AT,2000,A61C,6.535650,Medical_precision_and_optical_instruments,high_tech,59.625778,183.423946,20260.111317,75042.868275,1.204053,0.092581
2,AT,2000,A61F,2.757880,Medical_precision_and_optical_instruments,high_tech,59.625778,183.423946,20260.111317,75042.868275,1.204053,0.092581
3,AT,2000,A61H,0.333300,Medical_precision_and_optical_instruments,high_tech,59.625778,183.423946,20260.111317,75042.868275,1.204053,0.092581
4,AT,2000,A61K,9.507566,Pharmaceuticals,high_tech,46.606193,183.423946,10331.638863,75042.868275,1.845558,0.297150
...,...,...,...,...,...,...,...,...,...,...,...,...
11406,US,2007,H04Q,533.819102,Radio_TV_and_communications_equipment,high_tech,46449.384591,137335.006940,85202.770841,239495.205348,0.950696,-0.025275
11407,US,2007,H04R,649.169292,Radio_TV_and_communications_equipment,high_tech,46449.384591,137335.006940,85202.770841,239495.205348,0.950696,-0.025275
11408,US,2007,H04S,75.059331,Radio_TV_and_communications_equipment,high_tech,46449.384591,137335.006940,85202.770841,239495.205348,0.950696,-0.025275
11409,US,2007,H05G,38.046213,Medical_precision_and_optical_instruments,high_tech,37807.848600,137335.006940,60513.008492,239495.205348,1.089554,0.042858


In [32]:
import pandas as pd

# Assuming medium_tech_2000_2007 is already defined and contains the relevant data

# 1. Sum by country, sector_tech, and year to get country_sector_patents
country_sector_patents = medium_tech_2000_2007.groupby(['country', 'sector_tech', 'year'])['country_year_cpc_fraction'].sum().reset_index()
country_sector_patents.rename(columns={'country_year_cpc_fraction': 'country_sector_patents'}, inplace=True)

# 2. Sum by country and year to get country_total_patents
country_total_patents = medium_tech_2000_2007.groupby(['country', 'year'])['country_year_cpc_fraction'].sum().reset_index()
country_total_patents.rename(columns={'country_year_cpc_fraction': 'country_total_patents'}, inplace=True)

# 3. Sum by sector_tech and year to get world_sector_patents
world_sector_patents = medium_tech_2000_2007.groupby(['sector_tech', 'year'])['country_year_cpc_fraction'].sum().reset_index()
world_sector_patents.rename(columns={'country_year_cpc_fraction': 'world_sector_patents'}, inplace=True)

# 4. Sum by year to get world_total_patents
world_total_patents = medium_tech_2000_2007.groupby(['year'])['country_year_cpc_fraction'].sum().reset_index()
world_total_patents.rename(columns={'country_year_cpc_fraction': 'world_total_patents'}, inplace=True)

# Merge these aggregated columns back into the original medium_tech_2000_2007 dataframe
medium_tech_2000_2007 = pd.merge(medium_tech_2000_2007, country_sector_patents, on=['country', 'sector_tech', 'year'], how='left')
medium_tech_2000_2007 = pd.merge(medium_tech_2000_2007, country_total_patents, on=['country', 'year'], how='left')
medium_tech_2000_2007 = pd.merge(medium_tech_2000_2007, world_sector_patents, on=['sector_tech', 'year'], how='left')
medium_tech_2000_2007 = pd.merge(medium_tech_2000_2007, world_total_patents, on=['year'], how='left')

# Calculate RTA: (country_sector_patents / country_total_patents) / (world_sector_patents / world_total_patents)
medium_tech_2000_2007['RTA'] = (medium_tech_2000_2007['country_sector_patents'] / medium_tech_2000_2007['country_total_patents']) / (medium_tech_2000_2007['world_sector_patents'] / medium_tech_2000_2007['world_total_patents'])

# Calculate Normalized RTA: (RTA - 1) / (RTA + 1)
medium_tech_2000_2007['Normalized_RTA'] = (medium_tech_2000_2007['RTA'] - 1) / (medium_tech_2000_2007['RTA'] + 1)

# Optionally, save the RTA-calculated dataset to a CSV file
medium_tech_2000_2007.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery/medium_tech_2000_2007_RTA.csv', index=False)

# Display the first few rows to verify the RTA and Normalized RTA calculation
medium_tech_2000_2007


,country,year,CPC_code,country_year_cpc_fraction,sector_tech,tech_sech,country_sector_patents,country_total_patents,world_sector_patents,world_total_patents,RTA,Normalized_RTA
0,AT,2000,A01C,2.000000,Machinery_and_equipment,medium_tech,148.581580,250.841863,24694.017996,51434.511567,1.233752,0.104645
1,AT,2000,A01F,2.000000,Machinery_and_equipment,medium_tech,148.581580,250.841863,24694.017996,51434.511567,1.233752,0.104645
2,AT,2000,A01G,1.000000,Machinery_and_equipment,medium_tech,148.581580,250.841863,24694.017996,51434.511567,1.233752,0.104645
3,AT,2000,A21B,1.000000,Machinery_and_equipment,medium_tech,148.581580,250.841863,24694.017996,51434.511567,1.233752,0.104645
4,AT,2000,A21C,1.500000,Machinery_and_equipment,medium_tech,148.581580,250.841863,24694.017996,51434.511567,1.233752,0.104645
...,...,...,...,...,...,...,...,...,...,...,...,...
24296,US,2007,H02P,244.264353,Electrical_machinery_and_apparatus,medium_tech,7698.311348,56650.038828,15032.488207,106033.506925,0.958534,-0.021172
24297,US,2007,H05B,772.573035,Machinery_and_equipment,medium_tech,27963.784592,56650.038828,48696.230014,106033.506925,1.074839,0.036070
24298,US,2007,H05C,5.724000,Electrical_machinery_and_apparatus,medium_tech,7698.311348,56650.038828,15032.488207,106033.506925,0.958534,-0.021172
24299,US,2007,H05F,10.361100,Machinery_and_equipment,medium_tech,27963.784592,56650.038828,48696.230014,106033.506925,1.074839,0.036070
